In [ ]:
from petastorm import make_reader
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis,skew,pearsonr
from scipy.signal import periodogram
from scipy.integrate import simpson
from ecgdetectors import Detectors
import matplotlib.ticker as ticker
import pywt
from biosppy.signals import ecg 
from sklearn.metrics import confusion_matrix,auc
from matplotlib.widgets import TextBox, Button
import sys
from sklearn import metrics
import pandas as pd
import seaborn as sn
import os
import neurokit2 as nk
import biosppy
from numba import njit
sys.path.append(os.path.join(os.getcwd(), ".."))
from Metrics import TSD_cal as TSD
from Metrics import HurstExponent as Hurst
from shared_utils import Statistical_test_index as STTI
from Metrics import MoRE_index_2011 as MoRE_2011
from Metrics import ECG_Assess_2022
from Metrics import Fiducial_metrics,Non_Fiducial_metrics,Our_SQA_method
path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
path_csv_ref_label = "/workspaces/maitrise/data/20221006_physio_quality/set-a/REFERENCE.csv"

In [ ]:
###Data organization : Training dataset and Testing dataset + Reference labels

with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        if data.signal_quality == "unacceptable".encode():
            break
        else : 
            continue

ECG_lead = data.signal_names
print(ECG_lead)
fs = data.sampling_frequency
dico_ECG = {}
for i,j in zip(ECG_lead,range(12)):
    
    dico_ECG[i] = data.signal[:,j]
N = len(dico_ECG[ECG_lead[0]])
print(data)



In [ ]:
##Get true label
label_ref = pd.read_csv(path_csv_ref_label)
label_ref = label_ref.to_numpy()
Y = label_ref[:,1].copy()
Y_true = Y[Y.copy()!="unlabeled"]
X_true = label_ref[:,0].copy()
X_true = X_true[Y!="unlabeled"].astype(int)


In [ ]:
##Observation of one patients :

def plot_ECG_signal(signal,name,length= data.signal_length,fs = data.sampling_frequency):
     x = np.array(range(0,(len(signal))))
     x = x/fs    
     #fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,10))
     # ax[0].plot(x,signal)
     # ax[0].set_title(f"Full signal of Lead {name.decode('utf8')}")
     # ax[0].grid()
     plt.plot(x,signal)
     plt.title(f"Close up signal of Lead {name.decode('utf8')}")
     plt.grid()
     # if len(x) == data.signal_length:
     plt.xlim([0,3])
     # else :
     #      ax[1].set_xlim([0,x[-1]])
     plt.show()

ECG_signal = data.signal
ECG_lead = data.signal_names
fs = data.sampling_frequency
status = data.noun_id
dico_ECG = {}

for i,j in zip(ECG_lead,range(12)):
     dico_ECG[i] = ECG_signal[:,j]
     print(dico_ECG[i].shape)
     plot_ECG_signal(dico_ECG[i],i)
     
     

In [ ]:
###Some utilitary functions : 


def get_time_axis(sign_length,fs):
    x = np.linspace(0,int(sign_length/fs),sign_length)
    return x

def SDR_Quality_lead(SDR_dict_lead,name_lead):
    SDR_good_quality = {}
    SDR_medium_quality = {}
    SDR_bad_quality = {}
    for i in name_lead:
        if (SDR_dict_lead[i][0]<0.5 or SDR_dict_lead[i][0]>0.8):
            SDR_bad_quality[i] = SDR_dict_lead[i]
        elif (SDR_dict_lead[i][0]<0.6 and SDR_dict_lead[i][0]>0.5) or (SDR_dict_lead[i][0]<0.8 and SDR_dict_lead[i][0]>0.7):
            SDR_medium_quality[i] = SDR_dict_lead[i]
        else : 
            SDR_good_quality[i] = SDR_dict_lead[i]
    return SDR_good_quality,SDR_medium_quality,SDR_bad_quality


def wPMF_Quality_lead(wPMF_dict_lead,name_lead):
    wPMF_good_quality = {}
    wPMF_medium_quality = {}
    wPMF_bad_quality = {}
    for i in name_lead:
        if (wPMF_dict_lead[i][0]<0.25 or wPMF_dict_lead[i][0]>0):
            wPMF_bad_quality[i] = wPMF_dict_lead[i]
        elif (wPMF_dict_lead[i][0]<0.5 and wPMF_dict_lead[i][0]>0.25):
            wPMF_medium_quality[i] = wPMF_dict_lead[i]
        elif (wPMF_dict_lead[i][0]>0.5): 
            wPMF_good_quality[i] = wPMF_dict_lead[i]
    return wPMF_good_quality,wPMF_medium_quality,wPMF_bad_quality

def D_Quality_lead(D_dict_lead,name_lead):
    D_good_quality = {}
    D_medium_quality = {}
    D_bad_quality = {}
    for i in name_lead:
        if (D_dict_lead[i][0]<0.25 or D_dict_lead[i][0]>0):
            D_bad_quality[i] = D_dict_lead[i]
        elif (D_dict_lead[i][0]<0.75 and D_dict_lead[i][0]>0.5):
            D_medium_quality[i] = D_dict_lead[i]
        elif (D_dict_lead[i][0]>0.75): 
            D_good_quality[i] = D_dict_lead[i]
    return D_good_quality,D_medium_quality,D_bad_quality

def TSD_Quality_lead(TSD_dict_lead,name_lead):
    TSD_good_quality = {}
    TSD_medium_quality = {}
    TSD_bad_quality = {}
    for i in name_lead:
        if (TSD_dict_lead[i][0]>1.40):
            TSD_bad_quality[i] = TSD_dict_lead[i]
        elif (TSD_dict_lead[i][0]<1.40 and TSD_dict_lead[i][0]>1.25):
            TSD_medium_quality[i] = TSD_dict_lead[i]
        elif (TSD_dict_lead[i][0]<1.25): 
            TSD_good_quality[i] = TSD_dict_lead[i]
    return TSD_good_quality,TSD_medium_quality,TSD_bad_quality

def SNR_Quality_lead(SNR_dict_lead,name_lead):
    SNR_good_quality = {}
    SNR_medium_quality = {}
    SNR_bad_quality = {}
    for i in name_lead:
        if (SNR_dict_lead[i][0]<0.5):
            SNR_bad_quality[i] = SNR_dict_lead[i]
        elif (SNR_dict_lead[i][0]>0.5 and SNR_dict_lead[i][0]<50):
            SNR_medium_quality[i] = SNR_dict_lead[i]
        elif (SNR_dict_lead[i][0]>50): 
            SNR_good_quality[i] = SNR_dict_lead[i]
    return SNR_good_quality,SNR_medium_quality,SNR_bad_quality

def Morph_Quality_lead(M_dict_lead,name_lead):
    M_good_quality = {}
    M_medium_quality = {}
    M_bad_quality = {}
    for i in name_lead:
        if (M_dict_lead[i][0]<0.4):
            M_bad_quality[i] = M_dict_lead[i]
        elif (M_dict_lead[i][0]>0.4 and M_dict_lead[i][0]<0.66):
            M_medium_quality[i] = M_dict_lead[i]
        elif (M_dict_lead[i][0]>=0.66): 
            M_good_quality[i] = M_dict_lead[i]
    return M_good_quality,M_medium_quality,M_bad_quality

def CrossCor_Quality_lead(M_dict_lead,name_lead):
    M_good_quality = {}
    M_medium_quality = {}
    M_bad_quality = {}
    for i in name_lead:
        if (M_dict_lead[i][0]<0.4):
            M_bad_quality[i] = M_dict_lead[i]
        elif (M_dict_lead[i][0]>0.4 and M_dict_lead[i][0]<0.66):
            M_medium_quality[i] = M_dict_lead[i]
        elif (M_dict_lead[i][0]>=0.66): 
            M_good_quality[i] = M_dict_lead[i]
    return M_good_quality,M_medium_quality,M_bad_quality

def flat_Quality_lead(M_dict_lead,name_lead):
    f_good_quality = {}
    f_medium_quality = {}
    f_bad_quality = {}
    for i in name_lead:
        if (M_dict_lead[i][0]<0.2):
            f_good_quality[i] = M_dict_lead[i]
        elif (M_dict_lead[i][0]>0.2 and M_dict_lead[i][0]<0.5):
            f_medium_quality[i] = M_dict_lead[i]
        elif (M_dict_lead[i][0]>0.5): 
            f_bad_quality[i] = M_dict_lead[i]
    return f_good_quality,f_medium_quality,f_bad_quality

def Kurtosis_Quality_lead(K_dict_lead,name_lead):
    K_good_quality = {}
    K_medium_quality = {}
    K_bad_quality = {}
    for i in name_lead:
        if (K_dict_lead[i][0]<5):
            K_bad_quality[i] =K_dict_lead[i][1]
        elif (K_dict_lead[i][0]>=5 and K_dict_lead[i][0]<10):
            K_medium_quality[i] = K_dict_lead[i][1]
        elif (K_dict_lead[i][0]>=10): 
            K_good_quality[i] = K_dict_lead[i][1]
    return K_good_quality,K_medium_quality,K_bad_quality

def HR_quality_lead(HR_dict,name_lead):
    HR_good_quality = {}
    HR_pathological_lead = {}
    HR_bad_quality = {}
    for i in name_lead : 
        val_HR = np.mean(HR_dict[i][0])
        if (val_HR>24 and val_HR<=220):
            HR_good_quality[i] = HR_dict[i]
        elif (val_HR>220 and val_HR<450):
            HR_pathological_lead[i] = HR_dict[i]
        else : 
            HR_bad_quality[i] = HR_dict[i]
    return HR_good_quality,HR_pathological_lead,HR_bad_quality

def Hurst_Quality_lead(HurstD_dict_lead,name_lead):
    HurstD_good_quality = {}
    HurstD_medium_quality = {}
    HurstD_bad_quality = {}
    for i in name_lead:
        if (HurstD_dict_lead[i][0]>1.40):
            HurstD_bad_quality[i] = HurstD_dict_lead[i]
        elif (HurstD_dict_lead[i][0]<1.40 and HurstD_dict_lead[i][0]>1.25):
            HurstD_medium_quality[i] = HurstD_dict_lead[i]
        elif (HurstD_dict_lead[i][0]<1.25): 
            HurstD_good_quality[i] = HurstD_dict_lead[i]
    return HurstD_good_quality,HurstD_medium_quality,HurstD_bad_quality


def set_classification_status(func_name,index_score):
    if func_name == "SDR":
        return SDR_classification_status(index_score)
    elif func_name == "wPMF":
        return wPMF_classification_status(index_score)
    elif func_name == "TSD":
        return TSD_classification_status(index_score)
    elif func_name == "SNR":
        return SNR_classification_status(index_score)
    elif func_name == "HR":
        return HR_classification_status(index_score)
    elif func_name=="Hurst":
        return HurstD_classification_status(index_score)
    elif func_name=="Morph":
        return Morph_classification_status(index_score)
    elif func_name=="flat":
        return flat_classification_status(index_score)
    elif func_name == "CrossCor":
        return CrossCor_classification_status(index_score)
    elif func_name == "Double":
        return D_classification_status(index_score)
    elif func_name=="Kurtos":
        return K_classification_status(index_score)

def set_quality_lead(func_name,funct_dict_lead,name_lead):
    if func_name == "SDR":
        return SDR_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "wPMF":
        return wPMF_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "TSD":
        return TSD_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "SNR":
        return SNR_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "HR":
        return HR_quality_lead(funct_dict_lead,name_lead)
    elif func_name == "Hurst":
        return Hurst_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "Morph":
        return Morph_Quality_lead(funct_dict_lead,name_lead)
    elif func_name=="flat":
        return flat_Quality_lead(funct_dict_lead,name_lead)
    elif func_name=="CrossCor":
        return CrossCor_Quality_lead(funct_dict_lead,name_lead)
    elif func_name=="Double":
        return D_Quality_lead(funct_dict_lead,name_lead)
    elif func_name=="Kurtos":
        return Kurtosis_Quality_lead(funct_dict_lead,name_lead)


def flat_classification_status(index_score):
    if index_score<0.5:
        return "acceptable"
    else : 
        return "unacceptable"

def wPMF_classification_status(mean_wPMF):
    if (mean_wPMF>0.11):
        return "acceptable"
    else : 
        return "unacceptable"

def D_classification_status(mean_D):
    if (mean_D>=0.35):
        return "acceptable"
    else : 
        return "unacceptable"

def SDR_classification_status(mean_SDR):
    if (mean_SDR>0.5 and mean_SDR<0.8):
        return "acceptable"
    else : 
        return "unacceptable"

def TSD_classification_status(mean_TSD):
    if (mean_TSD<1.57):
        return "acceptable"
    else : 
        return "unacceptable"

def HurstD_classification_status(mean_HurstD):
    if (mean_HurstD<1.5):
        return "acceptable"
    else : 
        return "unacceptable"

def SNR_classification_status(mean_SNR):
    if mean_SNR>0.5:
        return "acceptable"
    else : 
        return "unacceptable"

def Morph_classification_status(mean_M):
    if mean_M>=0.66:
        return "acceptable"
    else : 
        return "unacceptable"

def HR_classification_status(mean_HR):
    if mean_HR>24 and mean_HR<450:
        return "acceptable"
    else : 
        return "unacceptable"


def K_classification_status(mean_K):
    if mean_K>=5:
        return "acceptable"
    else : 
        return "unacceptable"


def CrossCor_classification_status(mean_CC):
    if mean_CC>=0.5:
        return "acceptable"
    else : 
        return "unacceptable"

def Sorter_X_array(X_arr):
    index_sorted = np.argsort(X_arr)
    X_arr_sort = np.sort(X_arr)
    return X_arr_sort,index_sorted

In [ ]:
##Trial Kurtosis:

def Kurto_score(dico_signal,name_lead,fs):
    dico_K = {}
    ar_mean = np.array([])
    for i in name_lead:
        K = kurtosis(dico_signal[i])
        if np.isnan(K):
            K = 0
        dico_K[i] = (K,dico_signal[i])
        ar_mean = np.append(ar_mean,K)
    return dico_K,np.mean(ar_mean)

In [ ]:
###Morphological QRS complex (by QRS complex, it is all the PQRST wave)
####Rule : if average Pearson correlation,between the matching template and each beats detected, >0.66 => Acceptable. Else : Unacceptable



def is_segment_flatline(sig):
    cond = np.where(np.diff(sig.copy())!=0.0,np.nan,True)
    if (len(cond[cond==True])<0.70*len(sig)):
        return False
    else : 
        return True

def PQRST_template_extractor(ECG_signal,rpeaks):
    ##From the Biosspy function _extract_heartbeats
    R = np.sort(rpeaks)
    length = len(ECG_signal)
    templates = []
    newR = []

    for r in R:
        a = r-(np.median(np.diff(rpeaks,1))/2)
        if a < 0:
            continue
        b = r+(np.median(np.diff(rpeaks,1))/2)
        if b > length:
            break
            
        templates.append(ECG_signal[int(a):int(b)])
        newR.append(r)

    templates = np.array(templates)
    newR = np.array(newR, dtype="int")

    return templates, newR



def Morph_score(signals,name_lead,fs):
    QRS_lead = {}
    QRS_arr = np.array([])
    detect = Detectors(fs)
    for i in name_lead:
        r_peaks = detect.pan_tompkins_detector(signals[i])
        #is_segment_flatline(signals[i]) or 
        if len(r_peaks)<=2:
            QRS_lead[i] = (0,signals[i])
            QRS_arr = np.append(QRS_arr,0)
            continue
        else :
            template,_ =PQRST_template_extractor(signals[i],rpeaks = r_peaks)
            empty_index = np.array([],dtype = int)
            for ble in range(template.shape[0]):
                if template[ble].size==0:
                    empty_index = np.append(empty_index,ble)
            template = np.delete(template,empty_index,0)
            index_maxima = [np.argmax(template[w].copy()) for w in range(template.shape[0])]
            median_index = np.median(index_maxima.copy())
            templates_good = template[np.isclose(index_maxima.copy(),median_index,rtol=0.1)].copy()
            if templates_good.shape[0] == 0:
                QRS_lead[i] = (0,signals[i])
                QRS_arr = np.append(QRS_arr,0)
                continue
            
            sig_mean = templates_good[0].copy()
            for j in range(1,templates_good.shape[0]):
                if sig_mean.size != templates_good[j].size:
                    templates_good[j] = templates_good[j][:len(sig_mean)]
                sig_mean = np.add(sig_mean,templates_good[j].copy())
            
            sig = sig_mean/templates_good.shape[0]
            r_p = np.array([])
            for w in range(templates_good.shape[0]):
                beats = templates_good[w,:].copy()
                r_p = np.append(r_p,pearsonr(sig,beats)[0])
            QRS_lead[i] = (np.mean(r_p.copy()),signals[i])
            QRS_arr = np.append(QRS_arr,np.mean(r_p.copy()))
    return QRS_lead,np.mean(QRS_arr)



In [ ]:
###Index Creation : SDR 
### The label will be as follow : 0.8>mean(SDR of all lead) > 0.5 = Acceptable;mean(SDR of all lead) <0.5 or >0.8 = Unacceptable
##For each lead, we will return a mor eprecise classification based on the folloying rules
## SDR<0.5 or SDR>0.8 = Bad quality ; 0.6>SDR>0.5 or 0.8>SDR>0.7= Medium quality; 0.7>SDR>0.6 = Good quality

def SDR_score(signals,name_lead,fs):
    ##SDR coeff:
    SDR_lead = {}
    SDR_arr = np.array([])
    for i in name_lead:
        f,PSD = periodogram(signals[i],fs)
        QRS_signal_PSD = np.sum(PSD[np.logical_and(f>=5,f<=14)])
        ECG_tot = np.sum(PSD[np.logical_and(f>=5,f<=50)],dtype = np.float64)
        if ECG_tot == 0:
            ECG_tot = np.sum(np.abs(signals[i])**2)
            if ECG_tot ==0:
                ECG_tot = 2**63-1
        SDR_val = QRS_signal_PSD/ECG_tot
        SDR_lead[i] = (SDR_val,signals[i])
        SDR_arr = np.append(SDR_arr,SDR_val)
    return SDR_lead,np.mean(SDR_arr)



In [ ]:
###Index Creation : wPMF
### The label will be as follow : mean(SDR of all lead) > 0.5 = Acceptable;mean(SDR of all lead) <0.5 = Unacceptable
##For each lead, we will return a mor eprecise classification based on the folloying rule: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9281614/#B25



def Wavelet_coef(sig,name,lev):
    All_coeff = pywt.wavedec(sig,name,level = lev)

    CA = All_coeff[0]
    CD = All_coeff[1:len(All_coeff)]
    return CA,CD  


def Energy_L2(coeff):
    return np.sum(np.abs(coeff)**2, dtype = np.float64)

def wPMF_score(dico_signal,name_lead,fs):
    waveletname = 'db4'
    level_w = 9
    wPMF_lead = {}
    wPMF_arr = np.array([],dtype = np.float64)
    for i in name_lead:
        CA_w,CD_w = Wavelet_coef(dico_signal[i],waveletname,level_w)
        CD_w = np.array(CD_w,dtype = object)
        CA_w = np.array(CA_w,dtype = object)
        E = np.array([])
        for CD in range(level_w):
            E = np.append(E,Energy_L2(CD_w[-(CD+1)]))
        E = np.append(E,Energy_L2(CA_w[0]))
        Etot = np.sum(E,dtype = np.float64)
        if Etot == 0:
            Etot = Energy_L2(dico_signal[i][:int((2**level_w)-1)])
            if Etot ==0:
                Etot = 2**63-1
        p = E/Etot
        SQI_ECG = np.sum(p[3:6])
        wPMF_lead[i] = (SQI_ECG,dico_signal[i])
        wPMF_arr = np.append(wPMF_arr,SQI_ECG)
    return wPMF_lead, np.mean(wPMF_arr, dtype = np.float64)

In [ ]:
##Der new score : a tester : 

def score_wPMF_TSD(dico_signal,name_lead,fs):
    dico_results = {}
    Dico_TSD,_=TSD.TSD_index(dico_signal,name_lead,fs)
    Dico_wPMF,_ = wPMF_score(dico_signal,name_lead,fs)
    for final in name_lead:
        dico_results[final] = ((Dico_wPMF[final][0])/(Dico_TSD[final][0]),dico_signal[final])
    return dico_results,np.mean([dico_results[e][0] for e in name_lead])

In [ ]:
###Index Creation : SNR
### The label will be as follow : mean(SNR of all lead) > 0.5 = Acceptable;mean(SNR of all lead) <0.5 = Unacceptable
##For each lead, we will return a more eprecise classification based on the folloying rules
## SNR<0.5  = Bad quality ; 0.5>SNR<10= Medium quality; SNR>10 = Good quality

def SNR_index(dico_signal,name_lead,fs):
    SNR_lead = {}
    SNR_arr = np.array([],dtype = np.float64)
    for i in name_lead:
        f,PSD = periodogram(dico_signal[i],fs)
        Sig_PSD = np.sum(PSD[np.logical_and(f>2,f<=40)])
        LF_PSD = np.sum(PSD[np.logical_and(f>=0,f<=2)])
        HF_PSD = np.sum(PSD[np.logical_and(f>40,f<=250)])
        if (LF_PSD+HF_PSD == 0.0):
            SNR = Sig_PSD/(LF_PSD+HF_PSD+0.0001)
        else:
            SNR = Sig_PSD/(LF_PSD+HF_PSD)
        SNR_db = 10*np.log10(SNR)
        if np.isinf(SNR_db):
            SNR_db = -100
        SNR_lead[i] = (SNR_db,dico_signal[i])
        SNR_arr = np.append(SNR_arr,SNR_db)
    return SNR_lead,np.mean(SNR_arr)
        


In [ ]:
##Def flatline score
### The label will be as follow : mean(len(segment flatline)) > 0.5 * len(signal) = unacceptable;Else : Acceptable
# For each lead, we will return a more eprecise classification based on the folloying rules
## len(segment flatline)<0.2*len(signal)  = Excellent quality ; len(segment flatline)<0.5*len(signal)= Medium quality; len(segment flatline)>0.5*len(signal) = BAD quality

def flatline_score(dico_signals,name_lead,fs):
    flat_lead = {}
    flat_arr = np.array([],dtype = np.float64)
    for i in name_lead:
        cond = np.where(np.diff(dico_signals[i].copy())!=0.0,np.nan,True)
        score = len(cond[cond==True])/len(dico_signals[i])
        flat_lead[i] = (score,dico_signals[i])
        flat_arr = np.append(flat_arr,score)
    return flat_lead,np.mean(flat_arr)

In [ ]:
### Trial index Corr coeff between each Lead : 

def Corr_lead_score(dico_signal,name_lead,fs):
    # dico_template = {}
    dico_results = {}

    copy_name = name_lead.copy()
    X = np.empty([len(copy_name),len(dico_signal[copy_name[0]])])
    for i in range(len(copy_name)):
        X[i,:] = dico_signal[copy_name[i]]
    M = np.corrcoef(X)
    if M.size == 1:
        dico_results[name_lead[0]] = (1,dico_signal[name_lead[0]])
        return dico_results,1/12
    M_arr = np.array([])
    for j in range(len(copy_name)):
        val = (np.mean(np.abs(M[j,:])))
        if np.isnan(val):
            val = 0
        dico_results[copy_name[j]] = (val,dico_signal[copy_name[j]])
        M_arr = np.append(M_arr,val)
    return dico_results,np.mean(M_arr)

In [ ]:
dico_SQI = {"SDR":SDR_score,"wPMF":wPMF_score,"TSD":TSD.TSD_index,"SNR":SNR_index,"Hurst":Hurst.HurstD_index}

In [ ]:
##The general function to run statistical test : 

def Runner_statistic(func,name_func,path_peta,y_true,y):
    matrix = {}
    matrix["Y_True"] = y_true

    ##Dictionary lead quality for each patient with SDR
    lead_patient_history_func = {}

    Big_dataset = np.array([])
    Lead_dataset_score = np.empty([1000,12])
    X_predicted = np.array([])
    Y_predicted = np.array([])
    func_val_index = np.array([])
    ind = 0
    with make_reader(path_peta) as reader:
        for sample in reader:
            data = sample
            X_predicted = np.append(X_predicted,int(data.noun_id))
            ECG_lead = data.signal_names
            fs = data.sampling_frequency
            status = data.noun_id
        
            dico_ECG = {}

            for i,j in zip(ECG_lead,range(12)):
                dico_ECG[i] = data.signal[:,j]
                
            func_lead,func_index= func(dico_ECG,ECG_lead,fs)

            Big_dataset = np.append(Big_dataset,func_index)
            func_val_index = np.append(func_val_index,func_index)
            lead_good,lead_medium,lead_bad = set_quality_lead(name_func,func_lead,ECG_lead)
            lead_patient_history_func[status] = np.array([lead_good,lead_medium,lead_bad])
            
            varr  = np.array([func_lead[j][0] for j in ECG_lead])
            Lead_dataset_score[ind,:] = varr
            ind+=1
        
      
    for val in Big_dataset:
        prediction = set_classification_status(name_func,val)
        Y_predicted = np.append(Y_predicted,prediction)
    
    X_pred_sorted,ind_sort = Sorter_X_array(X_predicted)
    Y_predicted = Y_predicted[ind_sort]
    func_val_index = func_val_index[ind_sort]
    Y_predicted = Y_predicted[y!="unlabeled"]
    func_val_index = func_val_index[y!="unlabeled"]
    matrix["Y_predict"] = Y_predicted
    cm = confusion_matrix(y_true, Y_predicted)
    tp, tn, fn, fp = cm[0][0],cm[1][1],cm[0][1],cm[1][0]
    print("TP = ",tp)
    print("TN = ",tn)
    print("FP = ",fp)
    print("FN =",fn)
    Acc = (tp+tn)/(tp+tn+fp+fn)
    Prec = tp/(tp+fp)
    Recall = tp/(tp+fn)
    F1 = (2*Recall*Prec)/(Recall+Prec)
    print("Accuracy = ",Acc)
    print("Precision = ",Prec)
    print("Recall = ",Recall)
    print("F1 score = ",F1)

    ##Confusion matrix :
    df = pd.DataFrame(matrix, columns=['Y_True','Y_predict'])
    confusion = pd.crosstab(df['Y_True'], df['Y_predict'], rownames=['Actual'], colnames=['Predicted'],margins = True)
    sn.heatmap(confusion, annot=True,fmt='g')
    plt.title(f"Confusion Matrix for using the {name_func} index")
    plt.show()

    return Y_predicted,func_val_index,lead_patient_history_func,Lead_dataset_score,X_predicted

In [ ]:
##Test index SDR
Y_pred_SDR,SDR_val_index,lead_patients_history_SDR,Lead_dataset_SDR,X_SDR = Runner_statistic(SDR_score,"SDR",path_petastorm,Y_true,Y)

In [ ]:
##Test index Moprhological
Y_pred_M,M_val_index,lead_patients_history_M,Lead_dataset_M,X_M = Runner_statistic(Morph_score,"Morph",path_petastorm,Y_true,Y)

In [ ]:
##Test Kurtosis : 

Y_pred_K,K_val_index,lead_patients_history_K,Lead_dataset_K,X_K = Runner_statistic(Kurto_score,"Kurtos",path_petastorm,Y_true,Y)

In [ ]:
##Test index CrossCor
Y_pred_CC,CC_val_index,lead_patients_history_CC,Lead_dataset_CC,X_CC = Runner_statistic(Corr_lead_score,"CrossCor",path_petastorm,Y_true,Y)

In [ ]:
#Test index Flatline : 
Y_pred_f,f_val_index,lead_patients_history_f,Lead_dataset_f,X_f = Runner_statistic(flatline_score,"flat",path_petastorm,Y_true,Y)

In [ ]:
###Test index wPMF

Y_pred_wPMF,wPMF_val_index,lead_patients_history_wPMF,Lead_dataset_wPMF,X_wPMF =Runner_statistic(wPMF_score,"wPMF",path_petastorm,Y_true,Y)

In [ ]:
###Test index wPMF/TSD

Y_pred_D,D_val_index,lead_patients_history_D,Lead_dataset_D,X_D =Runner_statistic(score_wPMF_TSD,"Double",path_petastorm,Y_true,Y)

In [ ]:
##Test index Hurst
Y_pred_HD,HD_val_index,lead_patients_history_HD,Lead_dataset_HD,X_HD = Runner_statistic(Hurst.HurstD_index,"Hurst",path_petastorm,Y_true,Y)

In [ ]:
###Test index TSD
Y_pred_TSD,TSD_val_index,lead_patients_history_TSD,Lead_dataset_TSD,X_TSD =Runner_statistic(TSD.TSD_index_solo,"TSD",path_petastorm,Y_true,Y)

In [ ]:
###Get histogram of optimal segment length for both acceptable and unacceptable lead:

c_acceptable = np.array([])
c_unacceptable = np.array([])

with make_reader(path_petastorm) as reader:
        for sample in reader:
            data = sample
            if data.signal_quality == "acceptable".encode():
                ECG_lead = data.signal_names
                ECG_signal = data.signal
                fs = data.sampling_frequency
                dico_ECG = {}

                for j in range(12):
                    c = TSD.Interval_calculator_lead(ECG_signal[:,j],fs)
                    c_acceptable = np.append(c_acceptable,c)
            elif data.signal_quality == "unacceptable".encode(): 
                ECG_lead = data.signal_names
                ECG_signal = data.signal
                fs = data.sampling_frequency
                dico_ECG = {}

                for j in range(12):
                    c = TSD.Interval_calculator_lead(ECG_signal[:,j],fs)
                    c_unacceptable = np.append(c_unacceptable,c)
            else : 
                break

print(c_unacceptable)
h_acc, bins_acc = np.histogram(c_acceptable)
h_unacc, bins_unacc = np.histogram(c_unacceptable)

fig,ax = plt.subplots(nrows = 1, ncols = 2,figsize = (15,15))
ax[0].hist(c_acceptable,bins = bins_acc)
ax[0].set_xlabel("Optimal segment")
ax[0].set_ylabel("Frequencies")
ax[0].set_title("Histogram of optimal segment legnth for all lead of acceptbale ECG")
ax[0].grid()
ax[1].hist(c_unacceptable,bins = bins_unacc)
ax[1].set_xlabel("Optimal segment")
ax[1].set_ylabel("Frequencies")
ax[1].set_title("Histogram of optimal segment length for all lead of unacceptbale ECG")
ax[1].grid()      

                

In [ ]:
##repartition max min val I1 I2 ECG :
I1_max_acc = np.array([])
I2_max_acc = np.array([])
I1_max_unacc = np.array([])
I2_max_unacc = np.array([])
I1_min_acc = np.array([])
I2_min_acc = np.array([])
I1_min_unacc = np.array([])
I2_min_unacc = np.array([])
with make_reader(path_petastorm) as reader:
        for sample in reader:
            data = sample
            if data.signal_quality == "acceptable".encode():
                ECG_lead = data.signal_names
                ECG_signal = data.signal
                fs = data.sampling_frequency
                dico_ECG = {}

                for j in range(12):
                    I1t,I2t,_ = TSD.discrepancies_mean_curve(data.signal[:,j],fs,0.001,0.005)
                    I1_max_acc = np.append(I1_max_acc,np.amax(I1t))
                    I1_min_acc = np.append(I1_min_acc,np.amin(I1t))
                    I2_max_acc = np.append(I2_max_acc,np.amax(I2t))
                    I2_min_acc = np.append(I2_min_acc,np.amin(I2t))
                    
            elif data.signal_quality == "unacceptable".encode(): 
                ECG_lead = data.signal_names
                ECG_signal = data.signal
                fs = data.sampling_frequency
                dico_ECG = {}

                for j in range(12):
                    I1t,I2t,_ = TSD.discrepancies_mean_curve(data.signal[:,j],fs,0.001,0.005)
                    I1_max_unacc = np.append(I1_max_acc,np.amax(I1t))
                    I1_min_unacc = np.append(I1_min_acc,np.amin(I1t))
                    I2_max_unacc = np.append(I2_max_acc,np.amax(I2t))
                    I2_min_unacc = np.append(I2_min_acc,np.amin(I2t))
                    
            else : 
                break

counts_I1max_acc, bins_I1max_acc = np.histogram(I1_max_acc)
counts_I1max_unacc, bins_I1max_unacc = np.histogram(I1_max_unacc)
counts_I2max_acc, bins_I2max_acc = np.histogram(I2_max_acc)
counts_I2max_unacc, bins_I2max_unacc = np.histogram(I2_max_unacc)
counts_I2min_acc, bins_I2min_acc = np.histogram(I2_min_acc)
counts_I2min_unacc, bins_I2min_unacc = np.histogram(I2_min_unacc)
counts_I1min_acc, bins_I1min_acc = np.histogram(I1_min_acc)
counts_I1min_unacc, bins_I1min_unacc = np.histogram(I1_min_unacc)
fig,ax = plt.subplots(nrows = 4, ncols = 2,figsize = (20,15), constrained_layout=True)
fig.tight_layout(h_pad=5)
ax[0,0].hist(bins_I1max_acc[:-1],bins_I1max_acc,weights = counts_I1max_acc,edgecolor='black')
ax[0,0].set_xlabel("I1(c) Max value")
ax[0,0].set_ylabel("Frequencies")
ax[0,0].set_title("Histogram of maximum value of I1 for all lead of acceptbale ECG")
ax[0,0].grid()
ax[0,1].hist(bins_I1max_unacc[:-1],bins_I1max_unacc,weights = counts_I1max_unacc,edgecolor='black')
ax[0,1].set_xlabel("I1(c) Max value")
ax[0,1].set_ylabel("Frequencies")
ax[0,1].set_title("Histogram of maximum value of I1 for all lead of unacceptbale ECG")
ax[0,1].grid()
ax[1,0].hist(bins_I1min_acc[:-1],bins_I1min_acc,weights = counts_I1min_acc,edgecolor='black')
ax[1,0].set_ylabel("Frequencies")
ax[1,0].set_title("Histogram of minimum value of I1 for all lead of acceptbale ECG")
ax[1,0].grid()
ax[1,1].hist(bins_I1min_unacc[:-1],bins_I1min_unacc,weights = counts_I1min_unacc,edgecolor='black')
ax[1,1].set_xlabel("I1(c) Min value")
ax[1,1].set_ylabel("Frequencies")
ax[1,1].set_title("Histogram of minimum value of I1 for all lead of unacceptbale ECG")
ax[1,1].grid()
ax[2,0].hist(bins_I2max_acc[:-1],bins_I2max_acc,weights = counts_I2max_acc,edgecolor='black')
ax[2,0].set_xlabel("I2(c) Max value")
ax[2,0].set_ylabel("Frequencies")
ax[2,0].set_title("Histogram of maximum value of I2 for all lead of acceptbale ECG")
ax[2,0].grid()
ax[2,1].hist(bins_I2max_unacc[:-1],bins_I2max_unacc,weights = counts_I2max_unacc,edgecolor='black')
ax[2,1].set_xlabel("I2(c) Max value")
ax[2,1].set_ylabel("Frequencies")
ax[2,1].set_title("Histogram of maximum value of I2 for all lead of unacceptbale ECG")
ax[2,1].grid()
ax[3,0].hist(bins_I2min_acc[:-1],bins_I2min_acc,weights = counts_I2min_acc,edgecolor='black')
ax[3,0].set_xlabel("I2(c) Min value")
ax[3,0].set_ylabel("Frequencies")
ax[3,0].set_title("Histogram of minimum value of I2 for all lead of acceptbale ECG")
ax[3,0].grid()
ax[3,1].hist(bins_I2min_unacc[:-1],bins_I2min_unacc,weights = counts_I2min_unacc,edgecolor='black')
ax[3,1].set_xlabel("I2(c) Min value")
ax[3,1].set_ylabel("Frequencies")
ax[3,1].set_title("Histogram of minimum value of I2 for all lead of unacceptbale ECG")
ax[3,1].grid()
fig.subplots_adjust(top=0.90)        

In [ ]:
##Test index SNR

Y_pred_SNR,SNR_val_index,lead_patients_history_SNR,Lead_dataset_SNR,X_SNR =Runner_statistic(SNR_index,"SNR",path_petastorm,Y_true,Y)

In [ ]:
###Trail 2D dimensions plot : Check if combining 2 index can separate well signal quality assessment

def The_2D_plot_creator(Y1,Y2,correct_label,name_1,name_2,name_label_used = "Original",semilog = False):
    plt.figure()

    for lab,c in zip(["acceptable","unacceptable"],["r","b"]):
        Y1_sel = Y1[correct_label == lab].copy()
        Y2_sel = Y2[correct_label == lab].copy()
        plt.scatter(Y1_sel,Y2_sel,color = c,label = lab,alpha = 0.3)
    plt.legend(["acceptable","unacceptable"])
    plt.grid()
    plt.xlabel(name_1)
    plt.ylabel(name_2)
    if semilog : 
        plt.semilogy()
    plt.title(f"{name_1} VS {name_2} with the {name_label_used} labelisation" )
    plt.show()

###Let's plot!

# The_2D_plot_creator(SDR_val_index,wPMF_val_index,Y_true,"SDR","wPMF")
# The_2D_plot_creator(M_val_index,TSD_val_index,Y_true,"Morphological","TSD")
# The_2D_plot_creator(wPMF_val_index,HD_val_index,Y_true,"wPMF","Hurst Fractal D")
# The_2D_plot_creator(wPMF_val_index,SNR_val_index,Y_true,"wPMF","SNR")
# The_2D_plot_creator(SDR_val_index,SNR_val_index,Y_true,"SDR","SNR")
The_2D_plot_creator(TSD_val_index,wPMF_val_index,Y_true,"TSD","wPMF")

#The_2D_plot_creator(TSD_val_index,SNR_val_index,Y_true,"TSD","SNR",semilog =True)


In [ ]:
from mpl_toolkits import mplot3d
%matplotlib widget
def The_3D_plot_creator(Y1,Y2,Y3,correct_label,name_1,name_2,name_3,name_label_used = "Original",semilog = False):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    for lab,col in zip(["acceptable","unacceptable"],["r","b"]):
        Y1_sel = Y1[correct_label == lab].copy()
        Y2_sel = Y2[correct_label == lab].copy()
        Y3_sel = Y3[correct_label == lab].copy()
        ax.scatter3D(Y1_sel,Y2_sel,Y3_sel,color = col,label = lab,alpha = 0.3)
    ax.legend(["acceptable","unacceptable"])
    ax.grid()
    ax.set_xlabel(name_1)
    ax.set_ylabel(name_2)
    ax.set_zlabel(name_3)

    if semilog : 
        plt.semilogy()
    plt.title(f"{name_1} VS {name_2} VS {name_3} with the {name_label_used} labelisation" )
    plt.show()


In [ ]:
The_3D_plot_creator(TSD_val_index,M_val_index,wPMF_val_index,Y_true,"TSD","Morphological","wPMF")

In [ ]:
###Implementation HeartBeat detector ==> First Step of most SQA method in the litterature
###Goal of this method : Get the RR interval of each metho and check if BPM are in physiological range
### If pathological range ==> acceptable but with a the following message : "Suspicion of pathologie ==> Type of pathology"

###Frequency range considered:
### 24<f<300 : Acceptable (can be pathological but not so sure! need classification for that)
### 300<f<450 : Acceptbable but pathological (BPM with high risk of Fibrillation with multiple focal discharge)
### f>450 or f<24 : Unacceptable

def HR_index_calculator(dico_signal,name_lead,fs):
    RR_intervals_signal = {}
    mean_RR_interval = np.array([])
    x = get_time_axis(len(dico_signal[name_lead[0]]),fs)
    detect = Detectors(fs)
    for i in name_lead:
        r_peaks = detect.pan_tompkins_detector(dico_signal[i])
        r_sec = x[r_peaks]
        r_msec = r_sec*1000
        if len(r_msec) <=1:
            RR_intervals_signal[i] = (2,dico_signal[i])
            mean_RR_interval = np.append(mean_RR_interval,2)
        else:
            RR_bpm_interval = (60/(np.diff(r_msec)))*1000
            if np.mean(RR_bpm_interval)<24 or np.mean(RR_bpm_interval)>425:
                RR_intervals_signal[i] = (2,dico_signal[i])
                mean_RR_interval = np.append(mean_RR_interval,2)
            else : 
                RR_intervals_signal[i] = (np.std(RR_bpm_interval)/np.mean(RR_bpm_interval),dico_signal[i])
                mean_RR_interval = np.append(mean_RR_interval,np.std(RR_bpm_interval)/np.mean(RR_bpm_interval))
        #RR_intervals_signal[i] = (np.diff(r_sec)*1000,r_peaks,np.mean(r_sec))
        #mean_RR_interval = np.append(mean_RR_interval,np.mean(r_sec))
    return RR_intervals_signal,np.mean(mean_RR_interval)




In [ ]:
###Test HR classifiction : 
Y_pred_HR,HR_val_index,lead_patients_history_HR,Lead_dataset_HR,X_HR=Runner_statistic(HR_index_calculator,"HR",path_petastorm,Y_true,Y)

In [ ]:
###Bad prediction presentation : Seeing what went wrong with each method

def Derprinterlead(dict_quality,t,prediction,act,quality,patient_ind):
    for i in dict_quality : 
        if type(dict_quality[i][0]) == np.ndarray:
            val = np.mean(dict_quality[i][0])
            plt.figure() 
            plt.plot(t,dict_quality[i][1].copy())
            plt.title(f"Full signal of Lead {i.decode('utf8')} for patient {patient_ind}")
            plt.grid()
            #plt.xlim([2,4])
            plt.figtext(1, 0.7, "HR value = {0:.2f}".format(val))
            plt.figtext( 1, 0.6, "Label assigned = {0:.2f}".format(prediction))
            plt.figtext( 1, 0.5, "Quality = {0:.2f}".format(quality))
        else : 
            plt.figure() 
            plt.plot(t,dict_quality[i][1].copy())
            plt.title(f"Full signal of Lead {i.decode('utf8')} for patient {patient_ind}")
            plt.grid()
            #plt.xlim([2,4])
            plt.figtext(1, 0.7, "Index value = {0:.2f}".format(dict_quality[i][0]))
            plt.figtext(1, 0.6, f"Label assigned = {prediction}")
            plt.figtext(1, 0.5, f"True label = {act}")
            plt.figtext(1, 0.4, "Quality = {}".format(quality))



def bad_prediction_case(pred,actual,X_actual,history_lead_patient,name_lead = ECG_lead[0]):
    index_bad = (actual!=pred)
    patients_concerned = X_actual[index_bad]
    i = patients_concerned[3]
    string = str(i)
    good_quality,medium_quality,bad_quality = history_lead_patient[string.encode()]
    N = 5000
    t = get_time_axis(N,fs)
    Derprinterlead(good_quality,t,pred[np.where(X_actual==i)],actual[np.where(X_actual==i)],"good",i)
    Derprinterlead(medium_quality,t,pred[np.where(X_actual==i)],actual[np.where(X_actual==i)],"medium",i)
    Derprinterlead(bad_quality,t,pred[np.where(X_actual==i)],actual[np.where(X_actual==i)],"bad",i)


    
    

In [ ]:
###Test Hurst:

bad_prediction_case(Y_pred_TSD,Y_true,X_true,lead_patients_history_TSD)


In [ ]:
##Time for some ROC (it's about DRIVE, it's about POWER, We stay angry, We DEVOUR......)


def wPMF_classification_status_ROC(mean_wPMF,thresh):
    if (mean_wPMF>thresh):
        return "acceptable"
    else : 
        return "unacceptable"

def D_classification_status_ROC(mean_D,thresh):
    if (mean_D>thresh):
        return "acceptable"
    else : 
        return "unacceptable"

def K_classification_status_ROC(mean_K,thresh):
    if (mean_K>thresh):
        return "acceptable"
    else : 
        return "unacceptable"

def flatline_classification_status_ROC(mean_f,thresh):
    if (mean_f<thresh):
        return "acceptable"
    else : 
        return "unacceptable"

def SDR_classification_status_ROC(mean_SDR,thresh):
    if (mean_SDR>thresh[0] and mean_SDR<thresh[1]):
        return "acceptable"
    else : 
        return "unacceptable"

def Morph_classification_status_ROC(mean_Morph,thresh):
    if mean_Morph>=thresh:
        return "acceptable"
    else : 
        return "unacceptable"

def TSD_classification_status_ROC(mean_TSD,thresh):
    if (mean_TSD<thresh):
        return "acceptable"
    else : 
        return "unacceptable"

def HurstD_classification_status_ROC(mean_HurstD,thresh):
    if (mean_HurstD<thresh):
        return "acceptable"
    else : 
        return "unacceptable"

def SNR_classification_status_ROC(mean_SNR,thresh):
    if mean_SNR>thresh:
        return "acceptable"
    else : 
        return "unacceptable"

def CrossCorr_classification_status_ROC(mean_CrossCor,thresh):
    if mean_CrossCor>thresh:
        return "acceptable"
    else : 
        return "unacceptable"

def HR_classification_status_ROC(mean_HR,thresh):
    if mean_HR>=thresh:
        return "acceptable"
    else : 
        return "unacceptable"

def set_classification_status_ROC(func_name,index_score,threshold):
    if func_name == "SDR":
        return SDR_classification_status_ROC(index_score,threshold)
    elif func_name == "wPMF":
        return wPMF_classification_status_ROC(index_score,threshold)
    elif func_name == "TSD":
        return TSD_classification_status_ROC(index_score,threshold)
    elif func_name == "SNR":
        return SNR_classification_status_ROC(index_score,threshold)
    elif func_name == "Hurst":
        return HurstD_classification_status_ROC(index_score,threshold)
    elif func_name == "Morph":
        return Morph_classification_status_ROC(index_score,threshold)
    elif func_name == "flat":
        return flatline_classification_status_ROC(index_score,threshold)
    elif func_name == "CrossCor":
        return CrossCorr_classification_status_ROC(index_score,threshold)
    elif func_name == "Double":
        return D_classification_status_ROC(index_score,threshold)
    elif func_name == "HR":
        return HR_classification_status_ROC(index_score,threshold)
    elif func_name == "Kurtos":
        return K_classification_status_ROC(index_score,threshold)



def ROC_runner(name_func,y_true,y,Lead_Dataset,X_score):
    y_truecop = y_true.copy()
    
    y_truecop[y_true=="acceptable"] = 1
    y_truecop[y_true=="unacceptable"] = 0
    y_truecop = y_truecop.astype(int)
    _,ind_sort = Sorter_X_array(X_score)
    Y_score = Lead_Dataset
    FPR,TPR,thresholds = metrics.roc_curve(y_truecop.copy(),Y_score.copy(),pos_label = 1)
    PREC,REC,thresh_PR = metrics.precision_recall_curve(y_truecop.copy(),Y_score.copy(),pos_label = 1)
    auc = metrics.roc_auc_score(y_truecop.copy(),Y_score.copy())
    ACC = np.array([])
    
    for j in thresholds:
        matrix = {}
        matrix["Y_True"] = y_true
        Y_predicted = np.array([])
        for arr in Lead_Dataset:
            prediction = set_classification_status_ROC(name_func,arr,j)
            Y_predicted = np.append(Y_predicted,prediction)
        ##Conversion : 
        Y_predicted[Y_predicted == "acceptable"] = 1
        Y_predicted[Y_predicted == "unacceptable"] = 0
        ##Convert to int:
        Y_predicted = Y_predicted.astype(int)
        acc = metrics.accuracy_score(y_truecop,Y_predicted)
        ACC = np.append(ACC,acc)
    
    return TPR,FPR,REC[:-1],PREC[:-1],ACC,thresholds,thresh_PR,auc

def ROC_plot(true_PR,False_PR,AUC,index_tested):
    plt.figure()
    plt.plot(False_PR,true_PR,label = "AUC = "+str(AUC))

    # for t,i,j in zip(Threshold,False_PR,true_PR):
    #     plt.annotate("{0:.2f}".format(t),xy = (i,j))
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve for index {index_tested} ")
    plt.legend(loc=4)
    plt.grid()
    plt.show()

def RP_plot(Recall,Prec,index_tested):
    plt.figure()
    plt.plot(Recall,Prec)
    # for t,i,j in zip(Threshold,False_PR,true_PR):
    #     plt.annotate("{0:.2f}".format(t),xy = (i,j))
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Recall vs Precision Curve for index {} with AUC ={:.2f}".format(index_tested,auc(Recall,Prec)))
    plt.grid()
    plt.show()


def ACC_Threshod_plot(Acc,Threshold,index_tested):
    plt.figure()
    plt.plot(Threshold,Acc)
    # for t,i,j in zip(Threshold,False_PR,true_PR):
    #     plt.annotate("{0:.2f}".format(t),xy = (i,j))
    plt.xlabel("Threshold")
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy vs Threshold Curve for index {index_tested} ")
    plt.grid()
    plt.show()
    print(Threshold[np.argmax(Acc)])

def R_P_T_plot(Rec,Prec,Thresh,index_tested):
    plt.figure()
    fig, ax1 = plt.subplots()
    print(Thresh[np.argwhere(np.diff(np.sign(Prec - Rec))).flatten()])
    ax2 = ax1.twinx()
    ax1.plot(Thresh, Rec, 'g-')
    ax1.set_ylabel("Recall",color='g')
    ax2.plot(Thresh, Prec, 'b-')
    ax2.set_ylabel("Precision",color='b')
    ax1.set_xlabel("Threshold")
    plt.title(f"Recall and Precision curve in function of Threshold for index {index_tested} ")
    plt.grid()
    plt.show()

def TPR_FPR_P_T_plot(FPR,TPR,Thresh,index_tested):
    plt.figure()
    fig, ax1 = plt.subplots()
    print(Thresh[np.argwhere(np.diff(np.sign(TPR - FPR))).flatten()])
    ax2 = ax1.twinx()
    ax1.plot(Thresh, TPR, 'g-')
    ax1.set_ylabel("TPR",color='g')
    ax2.plot(Thresh, FPR, 'b-')
    ax2.set_ylabel("FPR",color='b')
    ax1.set_xlabel("Threshold")
    plt.title(f"TPR and FPR curve in function of Threshold for index {index_tested} ")
    plt.grid()
    plt.show()

def optimal_thresh_Gmean(FPR,TPR,Thresh,index_tested):
    Gmean = np.sqrt(TPR*(1-FPR))
    print("According to Gmean : ",Thresh[np.argmax(Gmean)])
    plt.figure()
    plt.plot(Thresh,Gmean)
    # for t,i,j in zip(Threshold,False_PR,true_PR):
    #     plt.annotate("{0:.2f}".format(t),xy = (i,j))
    plt.xlabel("Threshold")
    plt.ylabel("Gmean")
    plt.title("Gmean vs Threshold Curve for index SQA ")
    plt.grid()
    plt.show()


In [ ]:
##Test for TSD:
TPR_TSD,FPR_TSD,REC_TSD,PREC_TSD,ACC_TSD,TSD_thresh,TSD_T_PR,AUC_TSD = ROC_runner("TSD",Y_true,Y,TSD_val_index,X_TSD)
print(TSD_thresh[np.argmin(np.sqrt((1-TPR_TSD)**2-(FPR_TSD)**2))])
ROC_plot(TPR_TSD,FPR_TSD,AUC_TSD,"TSD")
RP_plot(REC_TSD,PREC_TSD,"TSD")
ACC_Threshod_plot(ACC_TSD,TSD_thresh,"TSD")
R_P_T_plot(REC_TSD,PREC_TSD,TSD_T_PR,"TSD")
TPR_FPR_P_T_plot(FPR_TSD,TPR_TSD,TSD_thresh,"TSD")
optimal_thresh_Gmean(FPR_TSD,TPR_TSD,TSD_thresh,"TSD")

In [ ]:
##Test for Kurtosis:
TPR_K,FPR_K,REC_K,PREC_K,ACC_K,K_thresh,K_T_PR,AUC_K = ROC_runner("Kurtos",Y_true,Y,K_val_index,X_K)
print(K_thresh[np.argmin(np.sqrt((1-TPR_K)**2-(FPR_K)**2))])
ROC_plot(TPR_K,FPR_K,AUC_K,"Kurtosis")
RP_plot(REC_K,PREC_K,"Kurtosis")
ACC_Threshod_plot(ACC_K,K_thresh,"Kurtosis")
R_P_T_plot(REC_K,PREC_K,K_T_PR,"Kurtosis")
TPR_FPR_P_T_plot(FPR_K,TPR_K,K_thresh,"Kurtosis")
optimal_thresh_Gmean(FPR_K,TPR_K,K_thresh,"Kurtosis")

In [ ]:
##Test for HR:
TPR_HR,FPR_HR,REC_HR,PREC_HR,ACC_HR,HR_thresh,HR_T_PR,AUC_HR = ROC_runner("HR",Y_true,Y,HR_val_index,X_HR)
print(HR_thresh[np.argmin(np.sqrt((1-TPR_HR)**2-(FPR_HR)**2))])
ROC_plot(TPR_HR,FPR_HR,AUC_HR,"HR")
RP_plot(REC_HR,PREC_HR,"HR")
ACC_Threshod_plot(ACC_HR,HR_thresh,"HR")
R_P_T_plot(REC_HR,PREC_HR,HR_T_PR,"HR")
TPR_FPR_P_T_plot(FPR_HR,TPR_HR,HR_thresh,"HR")

In [ ]:
##Test for double:
TPR_D,FPR_D,REC_D,PREC_D,ACC_D,D_thresh,D_T_PR,AUC_D = ROC_runner("Double",Y_true,Y,D_val_index,X_D)
print(D_thresh[np.argmin(np.sqrt((1-TPR_D)**2-(FPR_D)**2))])
ROC_plot(TPR_D,FPR_D,AUC_D,"Double")
RP_plot(REC_D,PREC_D,"Double")
ACC_Threshod_plot(ACC_D,D_thresh,"Double")
R_P_T_plot(REC_D,PREC_D,D_T_PR,"Double")
TPR_FPR_P_T_plot(FPR_D,TPR_D,D_thresh,"Double")
optimal_thresh_Gmean(FPR_D,TPR_D,D_thresh,"Double")

In [ ]:
##Test for Hurst:

TPR_HD,FPR_HD,REC_HD,PREC_HD,ACC_HD,TSD_thresh,TSD_T_PR,AUC_HD = ROC_runner("Hurst",Y_true,Y,TSD_thresh,HD_val_index,X_HD)
print(TSD_thresh[np.argmin(np.sqrt((1-TPR_HD)**2-(FPR_HD)**2))])
ROC_plot(TPR_HD,FPR_HD,"Hurst")
RP_plot(REC_HD,PREC_HD,"Hurst")
ACC_Threshod_plot(ACC_HD,TSD_thresh,"Hurst")
R_P_T_plot(REC_HD,PREC_HD,TSD_T_PR,"Hurst")

In [ ]:
##Test for Morphological:
TPR_M,FPR_M,REC_M,PREC_M,ACC_M,M_thresh,M_T_PR,AUC_M = ROC_runner("Morph",Y_true,Y,M_val_index,X_M)
print(M_thresh[np.argmin(np.sqrt((1-TPR_M)**2-(FPR_M)**2))])
ROC_plot(TPR_M,FPR_M,AUC_M,"Morph")
RP_plot(REC_M,PREC_M,"Morph")
ACC_Threshod_plot(ACC_M,M_thresh,"Morph")
R_P_T_plot(REC_M,PREC_M,M_T_PR,"Morph")
optimal_thresh_Gmean(FPR_M,TPR_M,M_thresh,"Morph")

In [ ]:
##Test for CrossCorr:
TPR_CC,FPR_CC,REC_CC,PREC_CC,ACC_CC,CC_thresh,CC_T_PR,AUC_CC = ROC_runner("CrossCor",Y_true,Y,CC_val_index,X_CC)
print(CC_thresh[np.argmin(np.sqrt((1-TPR_CC)**2-(FPR_CC)**2))])
ROC_plot(TPR_CC,FPR_CC,AUC_CC,"CrossCor")
RP_plot(REC_CC,PREC_CC,"CrossCor")
ACC_Threshod_plot(ACC_CC,CC_thresh,"CrossCor")
R_P_T_plot(REC_CC,PREC_CC,CC_T_PR,"CrossCor")
TPR_FPR_P_T_plot(FPR_CC,TPR_CC,CC_thresh,"CrossCor")
optimal_thresh_Gmean(FPR_CC,TPR_CC,CC_thresh,"CrossCor")

In [ ]:
##Test for wPMF:
TPR_wPMF,FPR_wPMF,REC_wPMF,PREC_wPMF,ACC_wPMF,wPMF_thresh,wPMF_T_PR,AUC_wPMF = ROC_runner("wPMF",Y_true,Y,wPMF_val_index,X_wPMF)
print(wPMF_thresh[np.argmin(np.sqrt((1-TPR_wPMF)**2-(FPR_wPMF)**2))])
ROC_plot(TPR_wPMF,FPR_wPMF,AUC_wPMF,"wPMF")
RP_plot(REC_wPMF,PREC_wPMF,"wPMF")
ACC_Threshod_plot(ACC_wPMF,wPMF_thresh,"wPMF")
R_P_T_plot(REC_wPMF,PREC_wPMF,wPMF_T_PR ,"wPMF")
optimal_thresh_Gmean(FPR_wPMF,TPR_wPMF,wPMF_thresh,"wPMF")

In [ ]:
##Test for Flatline:
TPR_f,FPR_f,REC_f,PREC_f,ACC_f,f_thresh,f_T_PR,AUC_f = ROC_runner("flat",Y_true,Y,f_val_index,X_f)
print(f_thresh[np.argmin(np.sqrt((1-TPR_f)**2-(FPR_f)**2))])
ROC_plot(TPR_f,FPR_f,AUC_f,"flat")
RP_plot(REC_f,PREC_f,"flat")
ACC_Threshod_plot(ACC_f,f_thresh,"flat")
R_P_T_plot(REC_f,PREC_f,f_T_PR,"Flatline")
optimal_thresh_Gmean(FPR_f,TPR_f,f_thresh,"Flatline")

In [ ]:
##Test SNR 
TPR_SNR,FPR_SNR,REC_SNR,PREC_SNR,ACC_SNR,SNR_thresh,SNR_T_PR,AUC_SNR = ROC_runner("SNR",Y_true,Y,SNR_val_index,X_SNR)
print(SNR_thresh[np.argmin(np.sqrt((1-TPR_SNR)**2-(FPR_SNR)**2))])
ROC_plot(TPR_SNR,FPR_SNR,AUC_SNR,"SNR")
RP_plot(REC_SNR,PREC_SNR,"SNR")
ACC_Threshod_plot(ACC_SNR ,SNR_thresh,"SNR")
R_P_T_plot(REC_SNR,PREC_SNR,SNR_T_PR,"SNR")
optimal_thresh_Gmean(FPR_SNR,TPR_SNR,SNR_thresh,"SNR")

In [ ]:
###SQA Method trial : HR then SNR then wPMF + SDR (When TSD is working, We will place TSD)
##Matrix of Regularity : 


def SQA_MoRE(dico_signal,name_lead,fs):
    M = MoRE_2011.MoRE(dico_signal,name_lead,fs)
    dico_results = M.MoRE_score()
    return dico_results,dico_results[name_lead[0]][2]

##Our SQA method:

def HR_score(signal,fs):
    detect = Detectors(fs)
    mean_RR_interval = 0
    x = get_time_axis(len(signal),fs)
    r_peaks = detect.pan_tompkins_detector(signal)
    if len(r_peaks)<=2:
        return 2
    r_sec = x[r_peaks]
    r_msec = r_sec*1000
    
    RR_bpm_interval = (60/(np.diff(r_msec)))*1000
    mean_RR_interval = np.mean(RR_bpm_interval)
    #SD_RR = np.std(RR_bpm_interval)
    if mean_RR_interval<24 or mean_RR_interval>420:
        return 10
    else : 
        return mean_RR_interval/np.std(RR_bpm_interval)
    #     return 1
        #RR_intervals_signal[i] = (np.diff(r_sec)*1000,r_peaks,np.mean(r_sec))
        #mean_RR_interval = np.append(mean_RR_interval,np.mean(r_sec))

def HR_score_dico(dico_signal,name_lead,fs):
    array_results = {}
    for i in name_lead:
        res = HR_score(dico_signal[i],fs)
        array_results[i] = (res,dico_signal[i])
    return array_results
#     dic_HR = {}
#     array_results = np.array([])
#     for i in name_lead:
#         res = HR_score(dico_signal[i],fs)
#         dic_HR[i] = (res,dico_signal[i])
#         array_results = np.append(array_results,res)
#     return dic_HR,np.mean(array_results)

def Morph_sig_score1(signal,fs):
    ##SDR coeff:
    detect = Detectors(fs)
    r_peaks = detect.pan_tompkins_detector(signal)
    template,_ = PQRST_template_extractor(signal,rpeaks = r_peaks)
    empty_index = np.array([],dtype = int)
    for ble in range(template.shape[0]):
        if template[ble].size==0:
            empty_index = np.append(empty_index,ble)
    template = np.delete(template,empty_index,0)
    index_maxima = np.array([np.argmax(template[w]) for w in range(template.shape[0])])
    median_index = np.median(index_maxima.copy())
    templates_good = template[np.isclose(index_maxima.copy(),median_index,rtol=0.1)].copy()
    if templates_good.size == 0:
        return 0
    sig_mean = templates_good[0]
    for i in range(1,templates_good.shape[0]):
        if sig_mean.size != templates_good[i].size:
            templates_good[i] = templates_good[i][:len(sig_mean)]
        sig_mean = np.add(sig_mean,templates_good[i].copy()) 
    
    sig = sig_mean/len(templates_good)
    r_p = np.array([])
    for t in templates_good:
        r_p = np.append(r_p,pearsonr(sig,t)[0])

    return np.mean(r_p)

def Morph_dico_score(dico_signal,name_lead,fs):
    array_results = np.array([])
    for i in name_lead:
        res = Morph_sig_score1(dico_signal[i],fs)
        if res>=0.4:
            array_results = np.append(array_results,1)
        else : 
            array_results = np.append(array_results,0)
    return array_results

def Flatline_dico_score(dico_signal,name_lead):
    array_results = np.array([])
    for i in name_lead:
        if flatline_score(dico_signal[i])>0:
            array_results = np.append(array_results,0)
        else : 
            array_results = np.append(array_results,1)
    return array_results

def wPMF_dico_score(dico_signal,name_lead,fs):
    array_results = np.array([])
    dico_res,_ = wPMF_score(dico_signal,name_lead,fs)
    for i in name_lead:
        if dico_res[i][0]>0.375:
            array_results = np.append(array_results,1)
        else : 
            array_results = np.append(array_results,0)
    return array_results

def qualification_status_selector(name_method,dico_results,name_signals,T):
    if name_method=="own":
        return set_qualification_SQA(dico_results,name_signals,T)
    elif name_method=="MoRE":
        return set_qualification_MoRE(dico_results,name_signals,T)

def set_qualification_MoRE(Dico_SQA,name_lead,T = 0.5):
    #indep_lead = np.array([name_lead[0],name_lead[1],name_lead[2],name_lead[6],name_lead[7],name_lead[8],name_lead[9],name_lead[10],name_lead[11]])
    results = np.array([])
    for i in name_lead:
        if Dico_SQA[i][0]>T:
            results = np.append(results,"unacceptable")
        else :
            results = np.append(results,"acceptable")
    #mean_results = np.mean([Dico_SQA[i][0] for i in name_lead])
    if np.mean([Dico_SQA[e][0]for e in name_lead])<T:#np.mean([Dico_SQA[e][0]for e in name_lead])>T:
        return "acceptable",results
    else : 
        return "acceptable",results


def set_qualification_SQA(dico_SQA,name_lead,T = 0.4):    #indep_lead = np.array([name_lead[0],name_lead[1],name_lead[2],name_lead[6],name_lead[7],name_lead[8],name_lead[9],name_lead[10],name_lead[11]])    results = np.array([])
    pathos = {}
    results = np.array([])
    for i in name_lead:
        if  dico_SQA[i][0]>T:
            results = np.append(results,"acceptable")
        else : 
            results = np.append(results,"unacceptable")

    mean_score_lead = np.mean(np.array([dico_SQA[i][0]for i in name_lead]))
    if mean_score_lead>T:
        return "unacceptable",results,pathos
    else : 
        return "acceptable",results,pathos

def set_qualification_NTSD(dico_SQA,name_lead,T=0.4):
    pathos = {}
    results = np.array([])
    for i in name_lead:
        if  dico_SQA[i][0]>T:
            results = np.append(results,"acceptable")
        else : 
            results = np.append(results,"unacceptable")

    mean_score_lead = np.mean(np.array([dico_SQA[i][0]for i in name_lead]))
    if mean_score_lead>T:
        return "unacceptable",results,pathos
    else : 
        return "acceptable",results,pathos


def Corr_dico_score(dico_signal,name_lead,fs):
    results = np.array([])
    dic,_  = Corr_lead_score(dico_signal,name_lead,fs)
    for i in name_lead:
        if dic[i][0]>=0.2:
            results = np.append(results,1)
        else : 
            results = np.append(results,0)
    return results

def SNR_dico_score(dico_signal,name_lead,fs):
    results = np.array([])
    dic,_  = SNR_index(dico_signal,name_lead,fs)
    for i in name_lead:
        if dic[i][0]>=0.5:
            results = np.append(results,1)
        else : 
            results = np.append(results,0)
    return results

def SQA_method_score(dico_signal,name_lead,fs):
    ###Scores Index : 
    dico_results = {}
    copy_name = name_lead.copy()
    # HR_lead = HR_score_dico(dico_signal,copy_name,fs)
    
    # if not HR_lead.all():
    #     HR_bad_lead  = copy_name[HR_lead == 0]
    #     for h in HR_bad_lead:
    #         dico_results[h] = (-100,dico_signal[h])
    #     copy_name = copy_name[HR_lead!=0]
    # if len(copy_name) == 0:
    #     return dico_results,np.mean(np.array([dico_results[e][0] for e in name_lead]))
    Dico_M,_ = Morph_score(dico_signal,copy_name,fs)
    Dico_CC,_ = Corr_lead_score(dico_signal,copy_name,fs)
    Dico_TSD,_=TSD.TSD_index_dico(dico_signal,copy_name,fs)
    Dico_HR = HR_score_dico(dico_signal,copy_name,fs)
    #Dico_wPMF,_ = wPMF_score(dico_signal,copy_name,fs)
    Dico_SNR,_ = SNR_index(dico_signal,copy_name,fs)
    for final in copy_name:
        val = (Dico_SNR[final][0]/Dico_HR[final][0])-Dico_TSD[final][0]*(Dico_M[final][0]*Dico_CC[final][0])
        dico_results[final] = (val,dico_signal[final])
    return dico_results,np.mean(np.array([dico_results[e][0] for e in name_lead]))

def SQA_NTSD_method_score(dico_signal,name_lead,fs):
    dico_results = {}
    copy_name = name_lead.copy()
    # HR_lead = HR_score_dico(dico_signal,copy_name,fs)
    
    # if not HR_lead.all():
    #     HR_bad_lead  = copy_name[HR_lead == 0]
    #     for h in HR_bad_lead:
    #         dico_results[h] = (-100,dico_signal[h])
    #     copy_name = copy_name[HR_lead!=0]
    # if len(copy_name) == 0:
    #     return dico_results,np.mean(np.array([dico_results[e][0] for e in name_lead]))
    # Dico_M,_ = Morph_score(dico_signal,copy_name,fs)
    # Dico_CC,_ = Corr_lead_score(dico_signal,copy_name,fs)
    Dico_SNR,_ = SNR_index(dico_signal,copy_name,fs)
    Dico_HR = HR_score_dico(dico_signal,copy_name,fs)
    for final in copy_name:
        #val_SNR = 10**(Dico_SNR[final][0]/(10))-(Dico_M[final][0]*Dico_CC[final][0])
        #val = ((val_SNR)*Dico_M[final][0]*Dico_CC[final][0])
        val  = (Dico_SNR[final][0]/Dico_HR[final][0])
        dico_results[final] = (val,dico_signal[final])
    return dico_results,np.mean(np.array([dico_results[e][0] for e in name_lead]))

def Select_Stat_run(name_method,path_peta,y_true,y):
    if name_method == "own":
        return Statistic_SQA(path_peta,y_true,y)
    elif name_method == "MoRE":
        return Statistic_MoRE(path_peta,y_true,y)
    elif name_method == "own_no_TSD":
        return Statistic_SQA_NTSD(path_peta,y_true,y)

def Statistic_MoRE(path_peta,y_true,y):

    matrix = {}
    matrix["Y_True"] = y_true

    ##Dictionary lead quality for each patient with SDR
    lead_patient_history_func = {}
    Lead_dataset_score = np.array([])
    X_predicted = np.array([])
    Y_predicted = np.array([])
    func_val_index = np.array([])
    with make_reader(path_peta) as reader:
        for sample in reader:
            data = sample
            X_predicted = np.append(X_predicted,int(data.noun_id))
            ECG_signal = data.signal
            ECG_lead = data.signal_names
            fs = data.sampling_frequency
            status = data.noun_id
        
            dico_ECG = {}

            for i,j in zip(ECG_lead,range(12)):
                dico_ECG[i] = ECG_signal[:,j]

            Dico_pred,value= MoRE_2011.MoRE_score(dico_ECG,ECG_lead,fs)
            prediction,pred_lead= set_qualification_MoRE(Dico_pred,ECG_lead)
            func_val_index = np.append(func_val_index,value)
            Y_predicted = np.append(Y_predicted,prediction)
            lead_patient_history_func[status] = np.array([Dico_pred,pred_lead],dtype=object)
            Lead_dataset_score = np.append(Lead_dataset_score,Dico_pred)
            

    X_pred_sorted,ind_sort = Sorter_X_array(X_predicted)
    Y_predicted = Y_predicted[ind_sort]
    Y_predicted = Y_predicted[y!="unlabeled"]
    func_val_index = func_val_index[ind_sort]
    func_val_index = func_val_index[y!="unlabeled"]
    matrix["Y_predict"] = Y_predicted
    cm = confusion_matrix(y_true, Y_predicted)
    tp, tn, fp, fn = cm[0][0],cm[1][1],cm[0][1],cm[1][0]
    print("TP = ",tp)
    print("TN = ",tn)
    print("FP = ",fp)
    print("FN =",fn)
    Acc = (tp+tn)/(tp+tn+fp+fn)
    Prec = tp/(tp+fp)
    Recall = tp/(tp+fn)
    F1 = (2*Recall*Prec)/(Recall+Prec)
    print("Accuracy = ",Acc)
    print("Precision = ",Prec)
    print("Recall = ",Recall)
    print("F1 score = ",F1)

    ##Confusion matrix :
    df = pd.DataFrame(matrix, columns=['Y_True','Y_predict'])
    confusion = pd.crosstab(df['Y_True'], df['Y_predict'], rownames=['Actual'], colnames=['Predicted'],margins = True)
    sn.heatmap(confusion, annot=True,fmt='g')
    plt.title(f"Confusion Matrix for using MoRE method")
    plt.show()

    return Y_predicted,lead_patient_history_func,Lead_dataset_score,X_predicted,func_val_index

def Statistic_SQA_NTSD(path_peta,y_true,y):

    matrix = {}
    matrix["Y_True"] = y_true

    ##Dictionary lead quality for each patient with SDR
    lead_patient_history_func = {}
    Lead_dataset_score = np.array([])
    X_predicted = np.array([])
    Y_predicted = np.array([])
    func_val_index = np.array([])
    with make_reader(path_peta) as reader:
        for sample in reader:
            data = sample
            X_predicted = np.append(X_predicted,int(data.noun_id))
            ECG_signal = data.signal
            ECG_lead = data.signal_names
            fs = data.sampling_frequency
            status = data.noun_id
        
            dico_ECG = {}

            for i,j in zip(ECG_lead,range(12)):
                dico_ECG[i] = ECG_signal[:,j]

            Dico_pred,value= SQA_NTSD_method_score(dico_ECG,ECG_lead,fs)
            prediction,pred_lead,pathos_lead= set_qualification_NTSD(Dico_pred,ECG_lead)
            func_val_index = np.append(func_val_index,value)
            Y_predicted = np.append(Y_predicted,prediction)
            lead_patient_history_func[status] = np.array([Dico_pred,pred_lead,pathos_lead],dtype = object)
            Lead_dataset_score = np.append(Lead_dataset_score,Dico_pred)
            
            

    X_pred_sorted,ind_sort = Sorter_X_array(X_predicted)
    Y_predicted = Y_predicted[ind_sort]
    Y_predicted = Y_predicted[y!="unlabeled"]
    func_val_index = func_val_index[ind_sort]
    func_val_index = func_val_index[y!="unlabeled"]
    matrix["Y_predict"] = Y_predicted
    cm = confusion_matrix(y_true, Y_predicted)
    tp, tn, fp, fn = cm[0][0],cm[1][1],cm[0][1],cm[1][0]
    print("TP = ",tp)
    print("TN = ",tn)
    print("FP = ",fp)
    print("FN =",fn)
    Acc = (tp+tn)/(tp+tn+fp+fn)
    Prec = tp/(tp+fp)
    Recall = tp/(tp+fn)
    F1 = (2*Recall*Prec)/(Recall+Prec)
    print("Accuracy = ",Acc)
    print("Precision = ",Prec)
    print("Recall = ",Recall)
    print("F1 score = ",F1)

    ##Confusion matrix :
    df = pd.DataFrame(matrix, columns=['Y_True','Y_predict'])
    confusion = pd.crosstab(df['Y_True'], df['Y_predict'], rownames=['Actual'], colnames=['Predicted'],margins = True)
    sn.heatmap(confusion, annot=True,fmt='g')
    plt.title(f"Confusion Matrix for using MoRE method")
    plt.show()

    return Y_predicted,lead_patient_history_func,Lead_dataset_score,X_predicted,func_val_index



def Statistic_SQA(path_peta,y_true,y):
    matrix = {}
    matrix["Y_True"] = y_true

    ##Dictionary lead quality for each patient with SDR
    lead_patient_history_func = {}
    Lead_dataset_score = np.array([])
    X_predicted = np.array([])
    Y_predicted = np.array([])
    func_val_index = np.array([])
    with make_reader(path_peta) as reader:
        for sample in reader:
            data = sample
            X_predicted = np.append(X_predicted,int(data.noun_id))
            ECG_signal = data.signal
            ECG_lead = data.signal_names
            fs = data.sampling_frequency
            status = data.noun_id
        
            dico_ECG = {}

            for i,j in zip(ECG_lead,range(12)):
                dico_ECG[i] = ECG_signal[:,j]

            Dico_pred,value= SQA_method_score(dico_ECG,ECG_lead,fs)
            func_val_index = np.append(func_val_index,value)
            prediction,pred_lead,pathos_lead = set_qualification_SQA(Dico_pred,ECG_lead)

            Y_predicted = np.append(Y_predicted,prediction)
            lead_patient_history_func[status] = np.array([Dico_pred,pred_lead,pathos_lead],dtype = object)
            Lead_dataset_score = np.append(Lead_dataset_score,Dico_pred)
            

    _,ind_sort = Sorter_X_array(X_predicted)
    Y_predicted = Y_predicted[ind_sort]
    func_val_index = func_val_index[ind_sort]
    func_val_index = func_val_index[y!="unlabeled"]
    Y_predicted = Y_predicted[y!="unlabeled"]
    matrix["Y_predict"] = Y_predicted
    cm = confusion_matrix(y_true, Y_predicted)
    tp, fp, fn, tn = cm[0][0],cm[0][1],cm[1][0],cm[1][1]
    print("TP = ",tp)
    print("TN = ",tn)
    print("FP = ",fp)
    print("FN =",fn)
    Acc = (tp+tn)/(tp+tn+fp+fn)
    Prec = tp/(tp+fp)
    Recall = tp/(tp+fn)
    F1 = (2*Recall*Prec)/(Recall+Prec)
    print("Accuracy = ",Acc)
    print("Precision = ",Prec)
    print("Recall = ",Recall)
    print("F1 score = ",F1)

    ##Confusion matrix :
    df = pd.DataFrame(matrix, columns=['Y_True','Y_predict'])
    confusion = pd.crosstab(df['Y_True'], df['Y_predict'], rownames=['Actual'], colnames=['Predicted'],margins = True)
    sn.heatmap(confusion, annot=True,fmt='g')
    plt.title(f"Confusion Matrix for using our SQA method")
    plt.show()

    return Y_predicted,lead_patient_history_func,Lead_dataset_score,X_predicted,func_val_index




In [ ]:
Y_SQA,Dataset_history,Lead_Dataset_SQA,X_SQA,SQA_val_index = Select_Stat_run("own",path_petastorm,Y_true,Y)

In [ ]:
Y_SQA_NTSD,Dataset_history_NTSD,Lead_Dataset_SQA_NTSD,X_SQA_NTSD,SQA_NTSD_val_index = Select_Stat_run("own_no_TSD",path_petastorm,Y_true,Y)

In [ ]:
Y_MoRE,Dataset_history_MoRE,Lead_Dataset_MoRE,X_MoRE,MoRE_val_index = Select_Stat_run("MoRE",path_petastorm,Y_true,Y)

In [ ]:
##ROC for our SQA : 
def ROC_SQA(path_peta,y_true,y,Threshold_tab,L_dataset,X_score):
    y_truecop = y_true.copy()
    
    y_truecop[y_true=="acceptable"] = 1
    y_truecop[y_true=="unacceptable"] = 0
    y_truecop = y_truecop.astype(int)
    _,ind_sort = Sorter_X_array(X_score)
    Y_score = SQA_val_index
    FPR,TPR,thresholds = metrics.roc_curve(y_truecop.copy(),Y_score.copy(),pos_label = 1)
    PREC,REC,thresh_PR = metrics.precision_recall_curve(y_truecop.copy(),Y_score.copy(),pos_label = 1)
    F_score = (2*PREC*REC)/(PREC+REC)
    auc = metrics.roc_auc_score(y_truecop.copy(),Y_score.copy())
    ACC = np.array([])
    
    for j in thresholds:
        Y_predicted = np.array([])
        for arr in L_dataset:
            prediction,_,_ = set_qualification_SQA(arr,ECG_lead,T = j)
            Y_predicted = np.append(Y_predicted,prediction)
        
        ##Conversion : 
        Y_predicted[Y_predicted == "acceptable"] = 1
        Y_predicted[Y_predicted == "unacceptable"] = 0
        Y_predicted = Y_predicted[ind_sort]
        Y_predicted = Y_predicted[y!="unlabeled"]
        ##Convert to int:
        Y_predicted = Y_predicted.astype(int)
        acc = metrics.accuracy_score(y_truecop,Y_predicted)
        ACC = np.append(ACC,acc)
    return TPR,FPR,REC[:-1],PREC[:-1],ACC,thresholds,thresh_PR,auc,F_score[:-1]

def ROC_SQA_NTSD(path_peta,y_true,y,Threshold_tab,L_dataset,X_score):
    y_truecop = y_true.copy()
    
    y_truecop[y_true=="acceptable"] = 1
    y_truecop[y_true=="unacceptable"] = 0
    y_truecop = y_truecop.astype(int)
    _,ind_sort = Sorter_X_array(X_score)
    Y_score = SQA_NTSD_val_index
    FPR,TPR,thresholds = metrics.roc_curve(y_truecop.copy(),Y_score.copy(),pos_label = 1)
    PREC,REC,thresh_PR = metrics.precision_recall_curve(y_truecop.copy(),Y_score.copy(),pos_label = 1)
    F_score = (2*PREC*REC)/(PREC+REC)
    auc = metrics.roc_auc_score(y_truecop.copy(),Y_score.copy())
    ACC = np.array([])
    
    for j in thresholds:
        Y_predicted = np.array([])
        for arr in L_dataset:
            prediction,_,_ = set_qualification_NTSD(arr,ECG_lead,T = j)
            Y_predicted = np.append(Y_predicted,prediction)
        
        ##Conversion : 
        Y_predicted[Y_predicted == "acceptable"] = 1
        Y_predicted[Y_predicted == "unacceptable"] = 0
        Y_predicted = Y_predicted[ind_sort]
        Y_predicted = Y_predicted[y!="unlabeled"]
        ##Convert to int:
        Y_predicted = Y_predicted.astype(int)
        acc = metrics.accuracy_score(y_truecop,Y_predicted)
        ACC = np.append(ACC,acc)
    return TPR,FPR,REC[:-1],PREC[:-1],ACC,thresholds,thresh_PR,auc,F_score[:-1]

def ROC_MoRE(path_peta,y_true,y,Threshold_tab,L_dataset,X_score):
    y_truecop = y_true.copy()
    
    y_truecop[y_true=="acceptable"] = 1
    y_truecop[y_true=="unacceptable"] = 0
    y_truecop = y_truecop.astype(int)
    _,ind_sort = Sorter_X_array(X_score)
    Y_score = MoRE_val_index
    print(Y_score)
    FPR,TPR,thresholds = metrics.roc_curve(y_truecop.copy(),Y_score.copy(),pos_label = 1)
    PREC,REC,thresh_PR = metrics.precision_recall_curve(y_truecop.copy(),Y_score.copy(),pos_label = 1)
    auc = metrics.roc_auc_score(y_truecop.copy(),Y_score.copy())
    ACC = np.array([])
    
    for j in thresholds:
        Y_predicted = np.array([])
        for arr in L_dataset:
            prediction,_ = set_qualification_MoRE(arr,ECG_lead,T = j)
            Y_predicted = np.append(Y_predicted,prediction)
        
        ##Conversion : 
        Y_predicted[Y_predicted == "acceptable"] = 1
        Y_predicted[Y_predicted == "unacceptable"] = 0
        Y_predicted = Y_predicted[ind_sort]
        Y_predicted = Y_predicted[y!="unlabeled"]
        ##Convert to int:
        Y_predicted = Y_predicted.astype(int)
        acc = metrics.accuracy_score(y_truecop,Y_predicted)
        ACC = np.append(ACC,acc)
    return TPR,FPR,REC[:-1],PREC[:-1],ACC,thresholds,thresh_PR,auc

def Choose_ROC(name_method,path_peta,y_true,y,Threshold_tab,L_dataset,X_score):
    if name_method == "own":
        return ROC_SQA(path_peta,y_true,y,Threshold_tab,L_dataset,X_score)
    elif name_method == "MoRE":
        return ROC_MoRE(path_peta,y_true,y,Threshold_tab,L_dataset,X_score)
    elif name_method == "own_no_TSD":
        return ROC_SQA_NTSD(path_peta,y_true,y,Threshold_tab,L_dataset,X_score)


In [ ]:
SQA_thresh = np.linspace(0,3,400)
TPR_SQA,FPR_SQA,REC_SQA,PREC_SQA,ACC_SQA,SQA_thresh1,SQA_T_PR,AUC_SQA,F_SQA = Choose_ROC("own",path_petastorm,Y_true,Y,SQA_thresh,Lead_Dataset_SQA,X_SQA)
print(SQA_thresh1[np.argmin(np.sqrt((1-TPR_SQA)**2-(FPR_SQA)**2))])
ROC_plot(TPR_SQA,FPR_SQA,AUC_SQA,"SQA")
RP_plot(REC_SQA,PREC_SQA,"SQA")
ACC_Threshod_plot(ACC_SQA,SQA_thresh1,"SQA")
R_P_T_plot(REC_SQA,PREC_SQA,SQA_T_PR,"SQA")
#TPR_FPR_P_T_plot(TPR_SQA,FPR_SQA,SQA_thresh1,"SQA")
Gmean = np.sqrt(TPR_SQA*(1-FPR_SQA))
print("According to Gmean : ",SQA_T_PR[np.argmax(F_SQA)])
plt.figure()
plt.plot(SQA_T_PR,F_SQA)
    # for t,i,j in zip(Threshold,False_PR,true_PR):
    #     plt.annotate("{0:.2f}".format(t),xy = (i,j))
plt.xlabel("Threshold")
plt.ylabel("Gmean")
plt.title("Gmean vs Threshold Curve for index SQA ")
plt.grid()
plt.show()
print()

In [ ]:
SQA_thresh = np.linspace(0,3,400)
TPR_SQA_NTSD,FPR_SQA_NTSD,REC_SQA_NTSD,PREC_SQA_NTSD,ACC_SQA_NTSD,SQA_thresh2,SQA_NTSD_T_PR,AUC_SQA_NTSD,F_SQA_NTSD = Choose_ROC("own_no_TSD",path_petastorm,Y_true,Y,SQA_thresh,Lead_Dataset_SQA_NTSD,X_SQA_NTSD)
print(SQA_thresh2[np.argmin(np.sqrt((1-TPR_SQA_NTSD)**2-(FPR_SQA_NTSD)**2))])
ROC_plot(TPR_SQA_NTSD,FPR_SQA_NTSD,AUC_SQA_NTSD,"SQA")
RP_plot(REC_SQA_NTSD,PREC_SQA_NTSD,"SQA")
ACC_Threshod_plot(ACC_SQA_NTSD,SQA_thresh2,"SQA")
R_P_T_plot(REC_SQA_NTSD,PREC_SQA_NTSD,SQA_NTSD_T_PR,"SQA no TSD")
#TPR_FPR_P_T_plot(TPR_SQA,FPR_SQA,SQA_thresh1,"SQA")
Gmean = np.sqrt(TPR_SQA_NTSD*(1-FPR_SQA_NTSD))
print("According to Gmean : ",SQA_thresh2[np.argmax(Gmean)])
plt.figure()
plt.plot(SQA_thresh2,Gmean)
    # for t,i,j in zip(Threshold,False_PR,true_PR):
    #     plt.annotate("{0:.2f}".format(t),xy = (i,j))
plt.xlabel("Threshold")
plt.ylabel("Gmean")
plt.title("Gmean vs Threshold Curve for index SQA ")
plt.grid()
plt.show()
print()

In [ ]:
MoRE_thresh = np.linspace(0,10,400)
TPR_MoRE,FPR_MoRE,REC_MoRE,PREC_MoRE,ACC_MoRE,MoRE_thresh1,MoRE_T_PR,AUC_MoRE = Choose_ROC("MoRE",path_petastorm,Y_true,Y,MoRE_thresh,Lead_Dataset_MoRE,X_MoRE)

ROC_plot(TPR_MoRE,FPR_MoRE,AUC_MoRE,"MoRE")
RP_plot(REC_MoRE,PREC_MoRE,"MoRE")
ACC_Threshod_plot(ACC_MoRE,MoRE_thresh1,"MoRE")
R_P_T_plot(REC_MoRE,PREC_MoRE,MoRE_T_PR,"MoRE")

In [ ]:
plt.figure()
    #auc_TSD = np.abs(simpson(TPR_TSD,x=FPR_TSD,dx = 1/(len(FPR_TSD))))
    #opt_thresh = Threshold[np.argmin(np.sqrt((1-true_PR)**2+(False_PR)**2))]
    #print(opt_thresh)
x = np.linspace(0,1,200)
auc_TSD = auc(FPR_TSD,TPR_TSD)
auc_SNR = auc(FPR_SNR,TPR_SNR)
auc_M = auc(FPR_M,TPR_M)
auc_CC = auc(FPR_CC,TPR_CC)
auc_wPMF = auc(FPR_wPMF,TPR_wPMF)
auc_f = auc(FPR_f,TPR_f)
auc_K = auc(FPR_K,TPR_K)
auc_SQA = auc(FPR_SQA,TPR_SQA)
auc_D = auc(FPR_D,TPR_D)
plt.plot(FPR_TSD,TPR_TSD,label = "TSD : AUC = {:.2f}".format(auc_TSD) )
plt.plot(FPR_SNR,TPR_SNR,label = "SNR : AUC = {:.2f}".format(auc_SNR) )
plt.plot(FPR_M,TPR_M,label = "Morphological : AUC = {:.2f}".format(auc_M))
plt.plot(FPR_CC,TPR_CC,label = "Correlation coef : AUC = {:.2f}".format(auc_CC))
plt.plot(FPR_wPMF,TPR_wPMF,label = "wPMF : AUC = {:.2f}".format(auc_wPMF))
plt.plot(FPR_f,TPR_f,label = "Flatline : AUC = {:.2f}".format(auc_f))
plt.plot(FPR_K,TPR_K,label = "Kurtosis : AUC = {:.2f}".format(auc_K))
plt.plot(FPR_D,TPR_D,label = "wPMF/TSD : AUC = {:.2f}".format(auc_D))
plt.plot(FPR_SQA,TPR_SQA,label = "SQA : AUC = {:.2f}".format(auc_SQA))
plt.plot(x,x,"--k",label = "Reference line")
    # for t,i,j in zip(Threshold,False_PR,true_PR):
    #     plt.annotate("{0:.2f}".format(t),xy = (i,j))
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC Curve for all indexes created ")
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.grid()
plt.show()

In [ ]:
##Precision recall Curve : 

plt.figure()
    #auc_TSD = np.abs(simpson(TPR_TSD,x=FPR_TSD,dx = 1/(len(FPR_TSD))))
    #opt_thresh = Threshold[np.argmin(np.sqrt((1-true_PR)**2+(False_PR)**2))]
    #print(opt_thresh)
auc_TSD = auc(REC_TSD,PREC_TSD)
auc_SNR = auc(REC_SNR,PREC_SNR)
auc_M = auc(REC_M,PREC_M)
auc_CC = auc(REC_CC,PREC_CC)
auc_wPMF = auc(REC_wPMF,PREC_wPMF)
auc_f = auc(REC_f,PREC_f)
auc_K = auc(REC_K,PREC_K)
auc_D = auc(REC_D,PREC_D)
auc_SQA = auc(REC_SQA,PREC_SQA)
plt.plot(REC_TSD,PREC_TSD,label = "TSD : AUC = {:.2f}".format(auc_TSD) )
plt.plot(REC_SNR,PREC_SNR,label = "SNR : AUC = {:.2f}".format(auc_SNR) )
plt.plot(REC_M,PREC_M,label = "Morphological : AUC = {:.2f}".format(auc_M))
plt.plot(REC_CC,PREC_CC,label = "Correlation coef : AUC = {:.2f}".format(auc_CC))
plt.plot(REC_wPMF,PREC_wPMF,label = "wPMF : AUC = {:.2f}".format(auc_wPMF))
plt.plot(REC_f,PREC_f,label = "Flatline : AUC = {:.2f}".format(auc_f))
plt.plot(REC_K,PREC_K,label = "Kurtosis : AUC = {:.2f}".format(auc_K))
plt.plot(REC_D,PREC_D,label = "wPMF/TSD : AUC = {:.2f}".format(auc_D))
plt.plot(REC_SQA,PREC_SQA,label = "SQA : AUC = {:.2f}".format(auc_SQA))
plt.plot(np.linspace(0,1,200),np.zeros(200),"--k",label = "Reference line")
    # for t,i,j in zip(Threshold,False_PR,true_PR):
    #     plt.annotate("{0:.2f}".format(t),xy = (i,j))
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title(f"PRecision Recall Curve for all indexes created ")
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.grid()
plt.show()

In [ ]:
##Use of Cross validation for determining optimal Threshold wPMF: 

wPMF_STTI = STTI.Statistic_reader(path_petastorm,Non_Fiducial_metrics.wPMF_score,"wPMF",[0,1])
wPMF_STTI.CrossValidation_index_opt_thresh()
wPMF_STTI.Optimal_threshold_calculator()
wPMF_STTI.plot_ROC_curve()
wPMF_STTI.plot_PR_curve()
wPMF_STTI.Accuracy_calculator()
wPMF_STTI.Plot_ROC_fold_graph()
wPMF_STTI.Plot_PR_fold_graph()

In [ ]:
SNR_STTI = STTI.Statistic_reader(path_petastorm,Non_Fiducial_metrics.SNR_index,"SNR",[-100,100])
SNR_STTI.CrossValidation_index_opt_thresh()
SNR_STTI.Optimal_threshold_calculator()
SNR_STTI.plot_ROC_curve()
SNR_STTI.plot_PR_curve()
SNR_STTI.Accuracy_calculator()
SNR_STTI.Plot_ROC_fold_graph()
SNR_STTI.Plot_PR_fold_graph()

In [ ]:
SNR_N_STTI = STTI.Statistic_reader(path_petastorm,Non_Fiducial_metrics.SNR_index,"SNR",[0,2],normalization= True)
SNR_N_STTI.CrossValidation_index_opt_thresh()
SNR_N_STTI.Optimal_threshold_calculator()
SNR_N_STTI.plot_ROC_curve()
SNR_N_STTI.plot_PR_curve()
SNR_N_STTI.Accuracy_calculator()
SNR_N_STTI.Plot_ROC_fold_graph()
SNR_N_STTI.Plot_PR_fold_graph()

In [ ]:
f_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.flatline_score,"Flatline",[0,5],normalization = True)
f_STTI.CrossValidation_index_opt_thresh()
f_STTI.Optimal_threshold_calculator()
f_STTI.plot_ROC_curve()
f_STTI.plot_PR_curve()
f_STTI.Accuracy_calculator()
f_STTI.Plot_ROC_fold_graph()
f_STTI.Plot_PR_fold_graph()

In [ ]:
TSD_STTI = STTI.Statistic_reader(path_petastorm,TSD.TSD_index,"TSD",[0,5],normalization = True)
TSD_STTI.CrossValidation_index_opt_thresh()
TSD_STTI.Optimal_threshold_calculator()
TSD_STTI.plot_ROC_curve()
TSD_STTI.plot_PR_curve()
TSD_STTI.Accuracy_calculator()
TSD_STTI.Plot_ROC_fold_graph()
TSD_STTI.Plot_PR_fold_graph()

In [ ]:
M_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.Morph_score,"Intralead",[-1,5])
M_STTI.CrossValidation_index_opt_thresh()
M_STTI.Optimal_threshold_calculator()
M_STTI.plot_ROC_curve()
M_STTI.plot_PR_curve()
M_STTI.Accuracy_calculator()
M_STTI.Plot_ROC_fold_graph()
M_STTI.Plot_PR_fold_graph()

In [ ]:
CC_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.Corr_lead_score,"Interlead",[-1,5])
CC_STTI.CrossValidation_index_opt_thresh()
CC_STTI.Optimal_threshold_calculator()
CC_STTI.plot_ROC_curve()
CC_STTI.plot_PR_curve()
CC_STTI.Accuracy_calculator()
CC_STTI.Plot_ROC_fold_graph()
CC_STTI.Plot_PR_fold_graph()

In [ ]:
K_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.Kurto_score,"Kurtosis",[0,30])
K_STTI.CrossValidation_index_opt_thresh()
K_STTI.Optimal_threshold_calculator()
K_STTI.plot_ROC_curve()
K_STTI.plot_PR_curve()
K_STTI.Accuracy_calculator()
K_STTI.Plot_ROC_fold_graph()
K_STTI.Plot_PR_fold_graph()

In [ ]:
SQA_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.SQA_method_score,"SQA",[0,2])
SQA_STTI.CrossValidation_index_opt_thresh()
SQA_STTI.Optimal_threshold_calculator()
SQA_STTI.plot_ROC_curve()
SQA_STTI.plot_PR_curve()
SQA_STTI.Accuracy_calculator()
SQA_STTI.Plot_ROC_fold_graph()
SQA_STTI.Plot_PR_fold_graph()

In [ ]:
SQA_SMOTE_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.SQA_SMOTE_method_score,"SQA_SMOTE",[0,2])
SQA_SMOTE_STTI.CrossValidation_index_opt_thresh()
SQA_SMOTE_STTI.Optimal_threshold_calculator()
SQA_SMOTE_STTI.plot_ROC_curve()
SQA_SMOTE_STTI.plot_PR_curve()
SQA_SMOTE_STTI.Accuracy_calculator()
SQA_SMOTE_STTI.Plot_ROC_fold_graph()
SQA_SMOTE_STTI.Plot_PR_fold_graph()

In [ ]:
##To check how the model evolve : 
SQA_STTI.print_prediction_model(40,0.55,[0,2])

In [ ]:
SQANTSD_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.SQA_NTSD_method_score,"SQA no TSD",[0,2])
SQANTSD_STTI.CrossValidation_index_opt_thresh()
SQANTSD_STTI.Optimal_threshold_calculator()
SQANTSD_STTI.plot_ROC_curve()
SQANTSD_STTI.plot_PR_curve()
SQANTSD_STTI.Accuracy_calculator()
SQANTSD_STTI.Plot_ROC_fold_graph()
SQANTSD_STTI.Plot_PR_fold_graph()

In [ ]:
##To check how the model evolve : 
SQANTSD_STTI.print_prediction_model(40,0.55,[0,2])

In [ ]:
SQANTSD_SMOTE_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.SQA_NTSD_SMOTE_method_score,"SQA no TSD SMOTE",[0,2])
SQANTSD_SMOTE_STTI.CrossValidation_index_opt_thresh()
SQANTSD_SMOTE_STTI.Optimal_threshold_calculator()
SQANTSD_SMOTE_STTI.plot_ROC_curve()
SQANTSD_SMOTE_STTI.plot_PR_curve()
SQANTSD_SMOTE_STTI.Accuracy_calculator()
SQANTSD_SMOTE_STTI.Plot_ROC_fold_graph()
SQANTSD_SMOTE_STTI.Plot_PR_fold_graph()

In [ ]:
##Model with feature selection made on SMOTE data : 
MAutoFeatsel_SMOTE_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.Model_ExtraTreeClassifier_SMOTED,"Auto Feature Selection Tree SMOTE",[0,2])
MAutoFeatsel_SMOTE_STTI.CrossValidation_index_opt_thresh()
MAutoFeatsel_SMOTE_STTI.Optimal_threshold_calculator()
MAutoFeatsel_SMOTE_STTI.plot_ROC_curve()
MAutoFeatsel_SMOTE_STTI.plot_PR_curve()
MAutoFeatsel_SMOTE_STTI.Accuracy_calculator()

In [ ]:
##Model with feature selection made on original data : 
MAutoFeatsel_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.Model_ExtraTreeClassifier,"Auto Feature Selection original Data",[0,2])
MAutoFeatsel_STTI.CrossValidation_index_opt_thresh()
MAutoFeatsel_STTI.Optimal_threshold_calculator()
MAutoFeatsel_STTI.plot_ROC_curve()
MAutoFeatsel_STTI.plot_PR_curve()
MAutoFeatsel_STTI.Accuracy_calculator()
MAutoFeatsel_STTI.Plot_ROC_fold_graph()
MAutoFeatsel_STTI.Plot_PR_fold_graph()

In [ ]:
##Model with feature selection made on original data : 
MRegu_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.Model_regularization,"L2 Regularization",[0,2])
MRegu_STTI.CrossValidation_index_opt_thresh()
MRegu_STTI.Optimal_threshold_calculator()
MRegu_STTI.plot_ROC_curve()
MRegu_STTI.plot_PR_curve()
MRegu_STTI.Accuracy_calculator()
MRegu_STTI.Plot_ROC_fold_graph()
MRegu_STTI.Plot_PR_fold_graph()

In [ ]:
HR_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.HR_index_calculator,"HR",[0,2],evaluation = "minimum")
HR_STTI.CrossValidation_index_opt_thresh()
HR_STTI.Optimal_threshold_calculator()
HR_STTI.plot_ROC_curve()
HR_STTI.plot_PR_curve()
HR_STTI.Accuracy_calculator()
HR_STTI.Plot_ROC_fold_graph()
HR_STTI.Plot_PR_fold_graph()

In [ ]:
MoRE_STTI = STTI.Statistic_reader(path_petastorm,MoRE_2011.MoRE_score,"MoRE",[0,2])
MoRE_STTI.CrossValidation_index_opt_thresh()
MoRE_STTI.Optimal_threshold_calculator()
MoRE_STTI.plot_ROC_curve()
MoRE_STTI.plot_PR_curve()
MoRE_STTI.Accuracy_calculator()
MoRE_STTI.Plot_ROC_fold_graph()
MoRE_STTI.Plot_PR_fold_graph()

In [ ]:
ECGASSESS_STTI = STTI.Statistic_reader(path_petastorm,ECG_Assess_2022.processing,"ECGASSESS",Y_true,[0,2],evaluation = "minimum")
ECGASSESS_STTI.CrossValidation_index_opt_thresh()
ECGASSESS_STTI.Optimal_threshold_calculator()
ECGASSESS_STTI.plot_ROC_curve()
ECGASSESS_STTI.plot_PR_curve()
ECGASSESS_STTI.Accuracy_calculator()

In [ ]:
##Now to get the overall Training and Testing PR and ROC curve for all the indexes : 

param_wPMF = wPMF_STTI._get_params()
param_SNR = SNR_STTI._get_params()
param_f = f_STTI._get_params()
param_SQA = SQA_STTI._get_params()
param_TSD = TSD_STTI._get_params()
param_HR = HR_STTI._get_params()
param_K = K_STTI._get_params()
param_M = M_STTI._get_params()
param_CC = CC_STTI._get_params()
param_NTSD = SQANTSD_STTI._get_params()
param_MoRE = MoRE_STTI._get_params()
#param_ECGASSESS = ECGASSESS_STTI._get_params()
get_name_dict = list(param_wPMF.keys())

name = ["wPMF","SNR_ECG","TSD","Flatline","HR","SQANTSD","Kurtosis","Intralead","Interlead","SQA","MoRE"]
dic_gen = {name[0] : param_wPMF,name[1]:param_SNR,name[2]:param_TSD,name[3]:param_f,name[4]:param_HR,name[5]:param_NTSD,name[6]:param_K,name[7]:param_M,name[8]:param_CC,name[9]:param_SQA,name[10]:param_MoRE}


fig,ax = plt.subplots(nrows = 2,ncols = 2,figsize= (15,15))

##Training Graphs : both PR and ROC :

x = np.linspace(0,1,200)
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(name))))

for na,i in zip(name,range(len(name))) : 
    c = next(color)
    FPR_train_ind = dic_gen[na][get_name_dict[2]][0]
    TPR_train_ind = dic_gen[na][get_name_dict[2]][1]
    SD_error_ROC_train = dic_gen[na][get_name_dict[2]][2]
    PREC_train_ind = dic_gen[na][get_name_dict[0]][0]
    REC_train_ind = dic_gen[na][get_name_dict[0]][1]
    SD_error_PR_train = dic_gen[na][get_name_dict[0]][2]

    FPR_test_ind = dic_gen[na][get_name_dict[3]][0]
    TPR_test_ind = dic_gen[na][get_name_dict[3]][1]
    SD_error_ROC_test = dic_gen[na][get_name_dict[3]][2]
    PREC_test_ind = dic_gen[na][get_name_dict[1]][0]
    REC_test_ind = dic_gen[na][get_name_dict[1]][1]
    SD_error_PR_test = dic_gen[na][get_name_dict[1]][2]

    auc_train_roc_ind = np.abs(np.trapz(TPR_train_ind,FPR_train_ind))
    auc_test_roc_ind = np.abs(np.trapz(TPR_test_ind,FPR_test_ind))
    auc_train_pr_ind = np.abs(np.trapz(PREC_train_ind,REC_train_ind))
    auc_test_pr_ind = np.abs(np.trapz(PREC_test_ind,REC_test_ind))

    # ax[0,0].errorbar(FPR_train_ind,TPR_train_ind,yerr=SD_error_ROC_train,label = "{} : AUC = {:.2f}".format(na,auc_train_roc_ind))
    # ax[1,0].errorbar(FPR_test_ind,TPR_test_ind,yerr = SD_error_ROC_test,label = "{} : AUC = {:.2f}".format(na,auc_test_roc_ind))
    # ax[0,1].errorbar(REC_train_ind,PREC_train_ind,yerr = SD_error_PR_train,label = "{} : AUC = {:.2f}".format(na,auc_train_pr_ind ))
    # ax[1,1].errorbar(REC_test_ind,PREC_test_ind,yerr = SD_error_PR_test,label = "{} : AUC = {:.2f}".format(na,auc_test_pr_ind))
    
    ax[0,0].plot(FPR_train_ind,TPR_train_ind,color = c,label = "{} : AUC = {:.2f}".format(na,auc_train_roc_ind))
    ax[1,0].plot(FPR_test_ind,TPR_test_ind,color = c,label = "{} : AUC = {:.2f}".format(na,auc_test_roc_ind))
    ax[0,1].plot(REC_train_ind,PREC_train_ind,color = c,label = "{} : AUC = {:.2f}".format(na,auc_train_pr_ind ))
    ax[1,1].plot(REC_test_ind,PREC_test_ind,color = c,label = "{} : AUC = {:.2f}".format(na,auc_test_pr_ind))



ax[0,0].plot(x,x,"--k",label = "Reference line")
ax[0,0].set_xlabel("False Positive Rate")
ax[0,0].set_ylabel("True Positive Rate")
ax[0,0].set_title(f"Training mean ROC Curve for all indexes created ")
ax[0,0].legend(loc = 4)
ax[0,0].grid()

ax[1,0].plot(x,x,"--k",label = "Reference line")
ax[1,0].set_xlabel("False Positive Rate")
ax[1,0].set_ylabel("True Positive Rate")
ax[1,0].set_title(f"Testing mean ROC Curve for all indexes created ")
ax[1,0].legend(loc = 4)
ax[1,0].grid()

ax[0,1].plot(x,np.zeros_like(x),"--k",label = "Reference line")
ax[0,1].set_xlabel("Recall")
ax[0,1].set_ylabel("Precision")
ax[0,1].set_title(f"Training mean PR Curve for all indexes created ")
ax[0,1].legend(loc = 4)
ax[0,1].grid()

ax[1,1].plot(x,np.zeros_like(x),"--k",label = "Reference line")
ax[1,1].set_xlabel("Recall")
ax[1,1].set_ylabel("Precision")
ax[1,1].set_title(f"Testing mean ROC Curve for all indexes created ")
ax[1,1].legend(loc = 4)
ax[1,1].grid()
#bbox_to_anchor=(1.04, 1), loc="upper left"
plt.show()





In [ ]:
##Generalized previous function : 

def ROC_PR_printer_class(dico_class):
    fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize= (15,10))
    ##Training Graphs : both PR and ROC : 
    x = np.linspace(0,1,200)
    color = iter(plt.cm.rainbow(np.linspace(0, 1, len(list(dico_class.keys())))))
    for na,i in zip(list(dico_class.keys()),range(len(list(dico_class.keys())))) : 
        c = next(color)
        FPR_test_ind = dic_gen[na][get_name_dict[3]][0]
        TPR_test_ind = dic_gen[na][get_name_dict[3]][1]
        PREC_test_ind = dic_gen[na][get_name_dict[1]][0]
        REC_test_ind = dic_gen[na][get_name_dict[1]][1]

        auc_test_roc_ind = np.abs(np.trapz(TPR_test_ind,FPR_test_ind))
        auc_test_pr_ind = np.abs(np.trapz(PREC_test_ind,REC_test_ind))
        ax[0].plot(FPR_test_ind,TPR_test_ind,color = c,label = "{} : AUC = {:.2f}".format(na,auc_test_roc_ind))
        ax[1].plot(REC_test_ind,PREC_test_ind,color = c,label = "{} : AUC = {:.2f}".format(na,auc_test_pr_ind))



    ax[0].plot(x,x,"--k",label = "Reference line")
    ax[0].set_xlabel("False Positive Rate")
    ax[0].set_ylabel("True Positive Rate")
    ax[0].set_title(f"Testing mean ROC Curve for all indexes created ")
    ax[0].legend(loc = 4)
    ax[0].grid()

    ax[1].plot(x,np.zeros_like(x),"--k",label = "Reference line")
    ax[1].set_xlabel("Recall")
    ax[1].set_ylabel("Precision")
    ax[1].set_title(f"Testing mean ROC Curve for all indexes created ")
    ax[1].legend(loc = 4)
    ax[1].grid()
    #bbox_to_anchor=(1.04, 1), loc="upper left"
    plt.show()


In [ ]:
param_SQA = SQA_STTI._get_params()
param_NTSD = SQANTSD_STTI._get_params()
param_MAutoFeatsel_SMOTE_STTI  = MAutoFeatsel_SMOTE_STTI._get_params()
param_MAutoFeatsel_STTI  = MAutoFeatsel_STTI._get_params()
params_SQASMOTE_STTI = SQA_SMOTE_STTI._get_params()
params_MRegu_STTI = MRegu_STTI._get_params()
params_NTSD_SMOTE = SQANTSD_SMOTE_STTI._get_params()
get_name_dict = list(param_SQA.keys())

name = ["SQA no TSD","SQA","Model ExtraTreeClassifier Feature SMOTE","Model ExtraTreeClassifier Feature Selection","SQA_SMOTE","L2_regularization","SQA no TSD SMOTE"]
dic_gen = {name[0] : param_NTSD,name[1]:param_SQA,name[2]:param_MAutoFeatsel_SMOTE_STTI ,name[3]:param_MAutoFeatsel_STTI,name[4]:params_SQASMOTE_STTI,name[5]:params_MRegu_STTI,name[6]:params_NTSD_SMOTE}
ROC_PR_printer_class(dic_gen)
